Source: https://medium.com/@ageitgey/natural-language-processing-is-fun-9a0bff37854e

In [13]:
!pip install en_core_web_sm

In [18]:
!pip install textacy

     |████████████████████████████████| 204kB 2.8MB/s 
     |████████████████████████████████| 143kB 58.0MB/s 
     |████████████████████████████████| 481kB 54.6MB/s 
     |████████████████████████████████| 3.0MB 48.9MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.7.2-cp36-cp36m-linux_x86_64.whl size=72988 sha256=cc6cd6249cca5aa2b310640f76944516d6fc118dee67087d905f4b9748291bf5
  Stored in directory: /root/.cache/pip/wheels/e8/fe/99/d8fa8f2ef7b82a625b0b77a84d319b0b50693659823c4effb4
  Created wheel for cytoolz: filename=cytoolz-0.10.1-cp36-cp36m-linux_x86_64.whl size=1256682 sha256=7b81bab95729c7894b947387b65338e6fd2550a28e2f5a7c556388a72048eb69
  Stored in directory: /root/.cache/pip/wheels/64/2a/18/d962b614e055577e7d9a3e4813e0742f822ca9c8800cc3783a
Successfully built jellyfish cytoolz


In [0]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [0]:
# The text we want to examine
text = """London is the capital and most populous city of England and 
the United Kingdom.  Standing on the River Thames in the south east 
of the island of Great Britain, London has been a major settlement 
for two millennia. It was founded by the Romans, who named it Londinium.
"""

# Parse the text with spaCy. This runs the entire pipeline.
doc = nlp(text)

# 'doc' now contains a parsed version of text. We can use it to do anything we want!
# For example, this will print out all the named entities that were detected:
for entity in doc.ents:
    print(f"{entity.text} ({entity.label_})")

London (GPE)
England (GPE)
the United Kingdom (GPE)
the River Thames (LOC)
Great Britain (GPE)
London (GPE)
two millennia (DATE)
Romans (NORP)
Londinium (LOC)


In [0]:
for token in doc:
    print(token.text)

**Going through thousands of documents and trying to redact all the names by hand could take years. But with NLP, it’s a breeze. Here’s a simple scrubber that removes all the names it detects:**

In [0]:
# Replace a token with "REDACTED" if it is a name
def replace_name_with_placeholder(token):
    if token.ent_iob != 0 and token.ent_type_ == "PERSON":
        return "[REDACTED] "
    else:
        return token.string

# Loop through all the entities in a document and check if they are names
def scrub(text):
    doc = nlp(text)
    for ent in doc.ents:
        ent.merge()
    tokens = map(replace_name_with_placeholder, doc)
    return "".join(tokens)

s = """
In 1950, Alan Turing published his famous article "Computing Machinery and Intelligence". In 1957, Noam Chomsky’s 
Syntactic Structures revolutionized Linguistics with 'universal grammar', a rule based system of syntactic structures.
"""

print(scrub(s))


In 1950, [REDACTED] published his famous article "Computing Machinery and Intelligence". In 1957, [REDACTED] 
Syntactic Structures revolutionized Linguistics with 'universal grammar', a rule based system of syntactic structures.



**There’s a python library called textacy that implements several common data extraction algorithms on top of spaCy. It’s a great starting point.
One of the algorithms it implements is called Semi-structured Statement Extraction. We can use it to search the parse tree for simple statements where the subject is “London” and the verb is a form of “be”. That should help us find facts about London.**

In [0]:
import spacy
import textacy.extract


# The text we want to examine
text = """London is the capital and most populous city of England and  the United Kingdom.  
Standing on the River Thames in the south east of the island of Great Britain, 
London has been a major settlement  for two millennia.  It was founded by the Romans, 
who named it Londinium.
"""

# Parse the document with spaCy
doc = nlp(text)

# Extract semi-structured statements
statements = textacy.extract.semistructured_statements(doc, "London")

for statement in statements:
    subject, verb, fact = statement
    print(f" - {fact}")

**Maybe that’s not too impressive. But if you run that same code on the entire London wikipedia article text instead of just three sentences, you’ll get this more impressive result:**

In [0]:
text='''

London is the capital and largest city of England and of the United Kingdom.[7][8] Standing on the River Thames in the south-east of England, at the head of its 50-mile (80 km) estuary leading to the North Sea, London has been a major settlement for two millennia. Londinium was founded by the Romans.[9] The City of London, London's ancient core − an area of just 1.12 square miles (2.9 km2) and colloquially known as the Square Mile − retains boundaries that closely follow its medieval limits.[10][11][12][13][14][note 1] The City of Westminster is also an Inner London borough holding city status. Greater London is governed by the Mayor of London and the London Assembly.[15][note 2][16]

London is considered to be one of the world's most important global cities[17][18][19] and has been termed the world's most powerful,[20] most desirable,[21] most influential,[22] most visited,[23] most expensive,[24][25] innovative,[26] sustainable,[27] most investment friendly,[28] and most popular for work[29] city. London exerts a considerable impact upon the arts, commerce, education, entertainment, fashion, finance, healthcare, media, professional services, research and development, tourism and transportation.[30][31] London ranks 26th out of 300 major cities for economic performance.[32] It is one of the largest financial centres[33] and has either the fifth or the sixth largest metropolitan area GDP.[note 3][34][35][36][37][38] It is the most-visited city as measured by international arrivals[39] and has the busiest city airport system as measured by passenger traffic.[40] It is the leading investment destination,[41][42][43][44] hosting more international retailers[45][46] and ultra high-net-worth individuals[47][48] than any other city. London's universities form the largest concentration of higher education institutes in Europe,[49] and London is home to highly ranked institutions such as Imperial College London in natural and applied sciences, the London School of Economics in social sciences, and the comprehensive University College London and King's College London.[50][51][52] In 2012, London became the first city to have hosted three modern Summer Olympic Games.[53]

London has a diverse range of people and cultures, and more than 300 languages are spoken in the region.[54] Its estimated mid-2018 municipal population (corresponding to Greater London) was 8,908,081,[4] the third most populous of any city in Europe[55] and accounts for 13.4% of the UK population.[56] London's urban area is the third most populous in Europe, after Moscow and Paris, with 9,787,426 inhabitants at the 2011 census.[57] The population within the London commuter belt is the most populous in Europe with 14,040,163 inhabitants in 2016.[note 4][3][58] London was the world's most populous city from c. 1831 to 1925.[59]

London contains four World Heritage Sites: the Tower of London; Kew Gardens; the site comprising the Palace of Westminster, Westminster Abbey, and St Margaret's Church; and the historic settlement in Greenwich where the Royal Observatory, Greenwich defines the Prime Meridian (0° longitude) and Greenwich Mean Time.[60] Other landmarks include Buckingham Palace, the London Eye, Piccadilly Circus, St Paul's Cathedral, Tower Bridge, Trafalgar Square and The Shard. London has numerous museums, galleries, libraries and sporting events. These include the British Museum, National Gallery, Natural History Museum, Tate Modern, British Library and West End theatres.[61] The London Underground is the oldest underground railway network in the world.

Main article: Etymology of London
"London" is an ancient name, already attested in the first century AD, usually in the Latinised form Londinium;[62] for example, handwritten Roman tablets recovered in the city originating from AD 65/70-80 include the word Londinio ("in London").[63]

Over the years, the name has attracted many mythicising explanations. The earliest attested appears in Geoffrey of Monmouth's Historia Regum Britanniae, written around 1136.[62] This had it that the name originated from a supposed King Lud, who had allegedly taken over the city and named it Kaerlud.[64]

Modern scientific analyses of the name must account for the origins of the different forms found in early sources: Latin (usually Londinium), Old English (usually Lunden), and Welsh (usually Llundein), with reference to the known developments over time of sounds in those different languages. It is agreed that the name came into these languages from Common Brythonic; recent work tends to reconstruct the lost Celtic form of the name as *[Londonjon] or something similar. This was adapted into Latin as Londinium and borrowed into Old English, the ancestor-language of English.[65]

The toponymy of the Common Brythonic form is much debated. A prominent explanation was Richard Coates' 1998 argument that the name derived from pre-Celtic Old European *(p)lowonida, meaning "river too wide to ford". Coates suggested that this was a name given to the part of the River Thames which flows through London; from this, the settlement gained the Celtic form of its name, *Lowonidonjon.[66] However, most work has accepted a Celtic origin for the name, and recent studies have favoured an explanation along the lines of a Celtic derivative of a Proto-Indo-European root *lendh- ('sink, cause to sink'), combined with the Celtic suffix *-injo- or *-onjo- (used to form place-names). Peter Schrijver has specifically suggested, on these grounds, that the name originally meant 'place that floods (periodically, tidally)'.[67][65]

Until 1889, the name "London" applied officially to the City of London, but since then it has also referred to the County of London and Greater London.[68] "London" is sometimes written informally as "LDN".[69][clarification needed]

History
Main articles: History of London and Timeline of London
Prehistory
In 1993, the remains of a Bronze Age bridge were found on the south foreshore, upstream of Vauxhall Bridge.[70] This bridge either crossed the Thames or reached a now lost island in it. Two of those timbers were radiocarbon dated to between 1750 BC and 1285 BC.[70]

In 2010, the foundations of a large timber structure, dated to between 4800 BC and 4500 BC,[71] were found on the Thames's south foreshore, downstream of Vauxhall Bridge.[72] The function of the mesolithic structure is not known. Both structures are on the south bank where the River Effra flows into the Thames.[72]

Roman London
Main article: Londinium

In 1300, the City was still confined within the Roman walls.
Although there is evidence of scattered Brythonic settlements in the area, the first major settlement was founded by the Romans about four years[1] after the invasion of AD 43.[73] This lasted only until around AD 61, when the Iceni tribe led by Queen Boudica stormed it, burning the settlement to the ground.[74] The next, heavily planned, incarnation of Londinium prospered, and it superseded Colchester as the capital of the Roman province of Britannia in 100. At its height in the 2nd century, Roman London had a population of around 60,000.[75]

Anglo-Saxon and Viking period London
With the collapse of Roman rule in the early 5th century, London ceased to be a capital, and the walled city of Londinium was effectively abandoned, although Roman civilisation continued in the area of St Martin-in-the-Fields until around 450.[76] From around 500, an Anglo-Saxon settlement known as Lundenwic developed slightly west of the old Roman city.[77] By about 680, the city had regrown into a major port, although there is little evidence of large-scale production. From the 820s repeated Viking assaults brought decline. Three are recorded; those in 851 and 886 succeeded, while the last, in 994, was rebuffed.[78]


The Lancastrian siege of London in 1471 is attacked by a Yorkist sally
The Vikings established Danelaw over much of eastern and northern England; its boundary stretched roughly from London to Chester. It was an area of political and geographical control imposed by the Viking incursions which was formally agreed by the Danish warlord, Guthrum and the West Saxon king Alfred the Great in 886. The Anglo-Saxon Chronicle recorded that Alfred "refounded" London in 886. Archaeological research shows that this involved abandonment of Lundenwic and a revival of life and trade within the old Roman walls. London then grew slowly until about 950, after which activity increased dramatically.[79]

By the 11th century, London was beyond all comparison the largest town in England. Westminster Abbey, rebuilt in the Romanesque style by King Edward the Confessor, was one of the grandest churches in Europe. Winchester had previously been the capital of Anglo-Saxon England, but from this time on, London became the main forum for foreign traders and the base for defence in time of war. In the view of Frank Stenton: "It had the resources, and it was rapidly developing the dignity and the political self-consciousness appropriate to a national capital."[80][81]

Middle Ages

Westminster Abbey, as seen in this painting (by Canaletto, 1749), is a World Heritage Site and one of London's oldest and most important buildings.
After winning the Battle of Hastings, William, Duke of Normandy was crowned King of England in the newly completed Westminster Abbey on Christmas Day 1066.[82] William constructed the Tower of London, the first of the many Norman castles in England to be rebuilt in stone, in the southeastern corner of the city, to intimidate the native inhabitants.[83] In 1097, William II began the building of Westminster Hall, close by the abbey of the same name. The hall became the basis of a new Palace of Westminster.[84][85]

In the 12th century, the institutions of central government, which had hitherto accompanied the royal English court as it moved around the country, grew in size and sophistication and became increasingly fixed in one place. For most purposes this was Westminster, although the royal treasury, having been moved from Winchester, came to rest in the Tower. While the City of Westminster developed into a true capital in governmental terms, its distinct neighbour, the City of London, remained England's largest city and principal commercial centre, and it flourished under its own unique administration, the Corporation of London. In 1100, its population was around 18,000; by 1300 it had grown to nearly 100,000.[86] Disaster struck in the form of the Black Death in the mid-14th century, when London lost nearly a third of its population.[87] London was the focus of the Peasants' Revolt in 1381.[88]

London was also a centre of England's Jewish population before their expulsion by Edward I in 1290. Violence against Jews took place in 1190, after it was rumoured that the new king had ordered their massacre after they had presented themselves at his coronation.[89] In 1264 during the Second Barons' War, Simon de Montfort's rebels killed 500 Jews while attempting to seize records of debts.[90]

Early modern

Map of London in 1593. There is only one bridge across the Thames, but parts of Southwark on the south bank of the river have been developed.
During the Tudor period the Reformation produced a gradual shift to Protestantism, and much of London property passed from church to private ownership, which accelerated trade and business in the city.[91] In 1475, the Hanseatic League set up its main trading base (kontor) of England in London, called the Stalhof or Steelyard. It existed until 1853, when the Hanseatic cities of Lübeck, Bremen and Hamburg sold the property to South Eastern Railway.[92] Woollen cloth was shipped undyed and undressed from 14th/15th century London to the nearby shores of the Low Countries, where it was considered indispensable.[93]

But the reach of English maritime enterprise hardly extended beyond the seas of north-west Europe. The commercial route to Italy and the Mediterranean Sea normally lay through Antwerp and over the Alps; any ships passing through the Strait of Gibraltar to or from England were likely to be Italian or Ragusan. Upon the re-opening of the Netherlands to English shipping in January 1565, there ensued a strong outburst of commercial activity.[94] The Royal Exchange was founded.[95] Mercantilism grew, and monopoly trading companies such as the East India Company were established, with trade expanding to the New World. London became the principal North Sea port, with migrants arriving from England and abroad. The population rose from an estimated 50,000 in 1530 to about 225,000 in 1605.[91]

In the 16th century William Shakespeare and his contemporaries lived in London at a time of hostility to the development of the theatre. By the end of the Tudor period in 1603, London was still very compact. There was an assassination attempt on James I in Westminster, in the Gunpowder Plot on 5 November 1605.[96]

In 1637, the government of Charles I attempted to reform administration in the area of London. The plan called for the Corporation of the City to extend its jurisdiction and administration over expanding areas around the City. Fearing an attempt by the Crown to diminish the Liberties of London, a lack of interest in administering these additional areas, or concern by city guilds of having to share power, the Corporation refused. Later called "The Great Refusal", this decision largely continues to account for the unique governmental status of the City.[97]


Vertue's 1738 plan of the Lines of Communication, built during the English Civil War
In the English Civil War the majority of Londoners supported the Parliamentary cause. After an initial advance by the Royalists in 1642, culminating in the battles of Brentford and Turnham Green, London was surrounded by a defensive perimeter wall known as the Lines of Communication. The lines were built by up to 20,000 people, and were completed in under two months.[98] The fortifications failed their only test when the New Model Army entered London in 1647,[99] and they were levelled by Parliament the same year.[100]

London was plagued by disease in the early 17th century,[101] culminating in the Great Plague of 1665–1666, which killed up to 100,000 people, or a fifth of the population.[102]


The Great Fire of London destroyed many parts of the city in 1666.
The Great Fire of London broke out in 1666 in Pudding Lane in the city and quickly swept through the wooden buildings.[103] Rebuilding took over ten years and was supervised by Robert Hooke[104][105][106] as Surveyor of London.[107] In 1708 Christopher Wren's masterpiece, St Paul's Cathedral was completed. During the Georgian era, new districts such as Mayfair were formed in the west; new bridges over the Thames encouraged development in South London. In the east, the Port of London expanded downstream. London's development as an international financial centre matured for much of the 1700s.

In 1762, George III acquired Buckingham House and it was enlarged over the next 75 years. During the 18th century, London was dogged by crime, and the Bow Street Runners were established in 1750 as a professional police force.[108] In total, more than 200 offences were punishable by death,[109] including petty theft.[110] Most children born in the city died before reaching their third birthday.[111]


View to the Royal Exchange in the City of London in 1886
The coffeehouse became a popular place to debate ideas, with growing literacy and the development of the printing press making news widely available; and Fleet Street became the centre of the British press. Following the invasion of Amsterdam by Napoleonic armies, many financiers relocated to London, especially a large Jewish community, and the first London international issue[clarification needed] was arranged in 1817. Around the same time, the Royal Navy became the world leading war fleet, acting as a serious deterrent to potential economic adversaries of the United Kingdom. The repeal of the Corn Laws in 1846 was specifically aimed at weakening Dutch economic power. London then overtook Amsterdam as the leading international financial centre.[112] In 1888, London became home to a series of murders by a man known only as Jack the Ripper and It has since become one of the world's most famous unsolved mysteries.

According to Samuel Johnson:

You find no man, at all intellectual, who is willing to leave London. No, Sir, when a man is tired of London, he is tired of life; for there is in London all that life can afford.

— Samuel Johnson, 1777[113]
Late modern and contemporary
London was the world's largest city from c.1831 to 1925,[59] with a population density of 325 people per hectare.[114] London's overcrowded conditions led to cholera epidemics,[115] claiming 14,000 lives in 1848, and 6,000 in 1866.[116] Rising traffic congestion led to the creation of the world's first local urban rail network. The Metropolitan Board of Works oversaw infrastructure expansion in the capital and some of the surrounding counties; it was abolished in 1889 when the London County Council was created out of those areas of the counties surrounding the capital.


British volunteer recruits in London, August 1914, during World War I

A bombed-out London street during the Blitz, World War II
London was bombed by the Germans during the First World War,[117] and during the Second World War, the Blitz and other bombings by the German Luftwaffe killed over 30,000 Londoners, destroying large tracts of housing and other buildings across the city.[118]

Immediately after the War, the 1948 Summer Olympics were held at the original Wembley Stadium, at a time when London was still recovering from the war.[119] From the 1940s onwards, London became home to many immigrants, primarily from Commonwealth countries such as Jamaica, India, Bangladesh and Pakistan,[120] making London one of the most diverse cities worldwide. In 1951, the Festival of Britain was held on the South Bank.[121] The Great Smog of 1952 led to the Clean Air Act 1956, which ended the "pea soup fogs" for which London had been notorious.[122]

Primarily starting in the mid-1960s, London became a centre for the worldwide youth culture, exemplified by the Swinging London subculture[123] associated with the King's Road, Chelsea[124] and Carnaby Street.[125] The role of trendsetter was revived during the punk era.[126] In 1965 London's political boundaries were expanded to take into account the growth of the urban area and a new Greater London Council was created.[127] During The Troubles in Northern Ireland, London was subjected to bombing attacks by the Provisional Irish Republican Army[128] for two decades, starting with the Old Bailey bombing in 1973.[129][130] Racial inequality was highlighted by the 1981 Brixton riot.[131]

Greater London's population declined steadily in the decades after the Second World War, from an estimated peak of 8.6 million in 1939 to around 6.8 million in the 1980s.[132] The principal ports for London moved downstream to Felixstowe and Tilbury, with the London Docklands area becoming a focus for regeneration, including the Canary Wharf development. This was borne out of London's ever-increasing role as a major international financial centre during the 1980s.[133] The Thames Barrier was completed in the 1980s to protect London against tidal surges from the North Sea.[134]

The Greater London Council was abolished in 1986, which left London without a central administration until 2000 when London-wide government was restored, with the creation of the Greater London Authority.[135] To celebrate the start of the 21st century, the Millennium Dome, London Eye and Millennium Bridge were constructed.[136] On 6 July 2005 London was awarded the 2012 Summer Olympics, making London the first city to stage the Olympic Games three times.[137] On 7 July 2005, three London Underground trains and a double-decker bus were bombed in a series of terrorist attacks.[138]

In 2008, Time named London alongside New York City and Hong Kong as Nylonkong, hailing it as the world's three most influential global cities.[139] In January 2015, Greater London's population was estimated to be 8.63 million, the highest level since 1939.[140] During the Brexit referendum in 2016, the UK as a whole decided to leave the European Union, but a majority of London constituencies voted to remain in the EU.[141]

Main articles: Local government in London, History of local government in London, and List of heads of London government
The administration of London is formed of two tiers: a citywide, strategic tier and a local tier. Citywide administration is coordinated by the Greater London Authority (GLA), while local administration is carried out by 33 smaller authorities.[142] The GLA consists of two elected components: the Mayor of London, who has executive powers, and the London Assembly, which scrutinises the mayor's decisions and can accept or reject the mayor's budget proposals each year. The headquarters of the GLA is City Hall, Southwark. The mayor since 2016 has been Sadiq Khan, the first Muslim mayor of a major Western capital.[143][144] The mayor's statutory planning strategy is published as the London Plan, which was most recently revised in 2011.[145] The local authorities are the councils of the 32 London boroughs and the City of London Corporation.[146] They are responsible for most local services, such as local planning, schools, social services, local roads and refuse collection. Certain functions, such as waste management, are provided through joint arrangements. In 2009–2010 the combined revenue expenditure by London councils and the GLA amounted to just over £22 billion (£14.7 billion for the boroughs and £7.4 billion for the GLA).[147]

The London Fire Brigade is the statutory fire and rescue service for Greater London. It is run by the London Fire and Emergency Planning Authority and is the third largest fire service in the world.[148] National Health Service ambulance services are provided by the London Ambulance Service (LAS) NHS Trust, the largest free-at-the-point-of-use emergency ambulance service in the world.[149] The London Air Ambulance charity operates in conjunction with the LAS where required. Her Majesty's Coastguard and the Royal National Lifeboat Institution operate on the River Thames,[150][151] which is under the jurisdiction of the Port of London Authority from Teddington Lock to the sea.[152]

National government
London is the seat of the Government of the United Kingdom. Many government departments, as well as the Prime Minister's residence at 10 Downing Street, are based close to the Palace of Westminster, particularly along Whitehall.[153] The British Parliament is often referred to as the "Mother of Parliaments" (although this sobriquet was first applied to England itself, and not to Westminster, by John Bright)[154] because it has been a model for many other parliamentary systems.[154] There are 73 Members of Parliament (MPs) from London, elected from local parliamentary constituencies in the national Parliament. As of May 2015, 49 are from the Labour Party, 21 are Conservatives, and three are Liberal Democrat.[155]

The UK government ministerial post of Minister for London was created in 1994 and currently occupied by Nick Hurd.[156]

Policing and crime
Main article: Crime in London
Policing in Greater London, with the exception of the City of London, is provided by the Metropolitan Police Service, overseen by the Mayor through the Mayor's Office for Policing and Crime (MOPAC).[157][158] The City of London has its own police force – the City of London Police.[159] The British Transport Police are responsible for police services on National Rail, London Underground, Docklands Light Railway and Tramlink services.[160] A fourth police force in London, the Ministry of Defence Police, do not generally become involved with policing the general public.

Crime rates vary widely by area, ranging from parts with serious issues to parts considered very safe. Today crime figures are made available nationally at Local Authority[161] and Ward level.[162] In 2015 there were 118 homicides, a 25.5% increase over 2014.[163] The Metropolitan Police have made detailed crime figures, broken down by category at borough and ward level, available on their website since 2000.[164]

Recorded crime has been rising in London, notably violent crime and murder by stabbing and other means have risen. There have been 50 murders from the start of 2018 to mid April 2018. Funding cuts to police in London are likely to have contributed to this, though other factors are also involved.[165]


Satellite view of London in June 2018
London, also referred to as Greater London, is one of nine regions of England and the top-level subdivision covering most of the city's metropolis.[note 5] The small ancient City of London at its core once comprised the whole settlement, but as its urban area grew, the Corporation of London resisted attempts to amalgamate the city with its suburbs, causing "London" to be defined in a number of ways for different purposes.[166]

Forty per cent of Greater London is covered by the London post town, within which 'LONDON' forms part of postal addresses.[167][168] The London telephone area code (020) covers a larger area, similar in size to Greater London, although some outer districts are excluded and some places just outside are included. The Greater London boundary has been aligned to the M25 motorway in places.[169]

Outward urban expansion is now prevented by the Metropolitan Green Belt,[170] although the built-up area extends beyond the boundary in places, resulting in a separately defined Greater London Urban Area. Beyond this is the vast London commuter belt.[171] Greater London is split for some purposes into Inner London and Outer London.[172] The city is split by the River Thames into North and South, with an informal central London area in its interior. The coordinates of the nominal centre of London, traditionally considered to be the original Eleanor Cross at Charing Cross near the junction of Trafalgar Square and Whitehall, are about 51°30′26″N 00°07′39″W.[173] However the geographical centre of London, on one definition, is in the London Borough of Lambeth, just 0.1 miles to the northeast of Lambeth North tube station.[174]

Status
Within London, both the City of London and the City of Westminster have city status and both the City of London and the remainder of Greater London are counties for the purposes of lieutenancies.[175] The area of Greater London includes areas that are part of the historic counties of Middlesex, Kent, Surrey, Essex and Hertfordshire.[176] London's status as the capital of England, and later the United Kingdom, has never been granted or confirmed officially—by statute or in written form.[note 6]

Its position was formed through constitutional convention, making its status as de facto capital a part of the UK's uncodified constitution. The capital of England was moved to London from Winchester as the Palace of Westminster developed in the 12th and 13th centuries to become the permanent location of the royal court, and thus the political capital of the nation.[180] More recently, Greater London has been defined as a region of England and in this context is known as London.[13]

Topography

London from Primrose Hill
Greater London encompasses a total area of 1,583 square kilometres (611 sq mi), an area which had a population of 7,172,036 in 2001 and a population density of 4,542 inhabitants per square kilometre (11,760/sq mi). The extended area known as the London Metropolitan Region or the London Metropolitan Agglomeration, comprises a total area of 8,382 square kilometres (3,236 sq mi) has a population of 13,709,000 and a population density of 1,510 inhabitants per square kilometre (3,900/sq mi).[181] Modern London stands on the Thames, its primary geographical feature, a navigable river which crosses the city from the south-west to the east. The Thames Valley is a floodplain surrounded by gently rolling hills including Parliament Hill, Addington Hills, and Primrose Hill. Historically London grew up at the lowest bridging point on the Thames. The Thames was once a much broader, shallower river with extensive marshlands; at high tide, its shores reached five times their present width.[182]

Since the Victorian era the Thames has been extensively embanked, and many of its London tributaries now flow underground. The Thames is a tidal river, and London is vulnerable to flooding.[183] The threat has increased over time because of a slow but continuous rise in high water level by the slow 'tilting' of the British Isles (up in Scotland and Northern Ireland and down in southern parts of England, Wales and Ireland) caused by post-glacial rebound.[184][185]

In 1974, a decade of work began on the construction of the Thames Barrier across the Thames at Woolwich to deal with this threat. While the barrier is expected to function as designed until roughly 2070, concepts for its future enlargement or redesign are already being discussed.[186]


London has a temperate oceanic climate (Köppen: Cfb ) receiving less precipitation than Rome, Bordeaux, Lisbon, Naples, Sydney or New York City.[187][188][189][190][191][192] Temperature extremes in London range from 38.1 °C (100.6 °F) at Kew during August 2003[193] down to −21.1 °C (−6.0 °F).[194] However, an unofficial reading of −24 °C (−11 °F) was reported on 3 January 1740.[195] Conversely, the highest unofficial temperature ever known to be recorded in the United Kingdom occurred in London in the 1808 heat wave. The temperature was recorded at 105 °F (40.6 °C) on 13 July. It is thought that this temperature, if accurate, is one of the highest temperatures of the millennium in the United Kingdom. It is thought that only days in 1513 and 1707 could have beaten this.[196] Since records began in London (first at Greenwich in 1841[197]), the warmest month on record is July 1868, with a mean temperature of 22.5 °C (72.5 °F) at Greenwich whereas the coldest month is December 2010, with a mean temperature of −6.7 °C (19.9 °F) at Northolt.[198] Records for atmospheric pressure have been kept at London since 1692. The highest pressure ever reported is 1,050 millibars (31 inHg) on 20 January 2020 and the lowest is 945.8 millibars (27.93 inHg) on 25 December 1821.[199][200]

Summers are generally warm, sometimes hot. London's average July high is 24 °C (74 °F). On average each year, London experiences 31 days above 25 °C (77.0 °F) and 4.2 days above 30.0 °C (86.0 °F) every year. During the 2003 European heat wave there were 14 consecutive days above 30 °C (86.0 °F) and 2 consecutive days when temperatures reached 38 °C (100 °F), leading to hundreds of heat-related deaths.[201] There was also a previous spell of 15 consecutive days above 32.2 °C (90.0 °F) in 1976 which also caused many heat related deaths.[202] The previous record high was 38 °C (100 °F) in August 1911 at the Greenwich station.[197] Droughts can also, occasionally, be a problem, especially in summer. Most recently in Summer 2018[203] and with much drier than average conditions prevailing from May to December.[204] However, the most consecutive days without rain was 73 days in the spring of 1893.[205]

Winters are generally cool with little temperature variation. Heavy snow is rare but snow usually happens at least once each winter. Spring and autumn can be pleasant. As a large city, London has a considerable urban heat island effect,[206] making the centre of London at times 5 °C (9 °F) warmer than the suburbs and outskirts. This can be seen below when comparing London Heathrow, 15 miles (24 km) west of London, with the London Weather Centre.[207]

Although London and the British Isles have a reputation of frequent rainfall, London's average of 602 millimetres (23.7 in) of precipitation annually actually makes it drier than the global average.[208] The absence of heavy winter rainfall leads to many climates around the Mediterranean having more annual precipitation than London.


Source #2: Met Office [213]
Districts
Main articles: List of districts of London and London boroughs
London's vast urban area is often described using a set of district names, such as Bloomsbury, Mayfair, Wembley and Whitechapel. These are either informal designations, reflect the names of villages that have been absorbed by sprawl, or are superseded administrative units such as parishes or former boroughs.

Such names have remained in use through tradition, each referring to a local area with its own distinctive character, but without official boundaries. Since 1965 Greater London has been divided into 32 London boroughs in addition to the ancient City of London.[214][215] The City of London is the main financial district,[216] and Canary Wharf has recently developed into a new financial and commercial hub in the Docklands to the east.

The West End is London's main entertainment and shopping district, attracting tourists.[217] West London includes expensive residential areas where properties can sell for tens of millions of pounds.[218] The average price for properties in Kensington and Chelsea is over £2 million with a similarly high outlay in most of central London.[219][220]

The East End is the area closest to the original Port of London, known for its high immigrant population, as well as for being one of the poorest areas in London.[221] The surrounding East London area saw much of London's early industrial development; now, brownfield sites throughout the area are being redeveloped as part of the Thames Gateway including the London Riverside and Lower Lea Valley, which was developed into the Olympic Park for the 2012 Olympics and Paralympics.[221]

Architecture
Main articles: Architecture of London, List of tallest buildings and structures in London, and List of demolished buildings and structures in London

The Tower of London, a medieval castle, dating in part to 1078

Trafalgar Square and its fountains, with Nelson's Column on the right
London's buildings are too diverse to be characterised by any particular architectural style, partly because of their varying ages. Many grand houses and public buildings, such as the National Gallery, are constructed from Portland stone. Some areas of the city, particularly those just west of the centre, are characterised by white stucco or whitewashed buildings. Few structures in central London pre-date the Great Fire of 1666, these being a few trace Roman remains, the Tower of London and a few scattered Tudor survivors in the City. Further out is, for example, the Tudor-period Hampton Court Palace, England's oldest surviving Tudor palace, built by Cardinal Thomas Wolsey c.1515.[222]

Part of the varied architectural heritage are the 17th-century churches by Wren, neoclassical financial institutions such as the Royal Exchange and the Bank of England, to the early 20th century Old Bailey and the 1960s Barbican Estate.

The disused – but soon to be rejuvenated – 1939 Battersea Power Station by the river in the south-west is a local landmark, while some railway termini are excellent examples of Victorian architecture, most notably St. Pancras and Paddington.[223] The density of London varies, with high employment density in the central area and Canary Wharf, high residential densities in inner London, and lower densities in Outer London.

The Monument in the City of London provides views of the surrounding area while commemorating the Great Fire of London, which originated nearby. Marble Arch and Wellington Arch, at the north and south ends of Park Lane, respectively, have royal connections, as do the Albert Memorial and Royal Albert Hall in Kensington. Nelson's Column is a nationally recognised monument in Trafalgar Square, one of the focal points of central London. Older buildings are mainly brick built, most commonly the yellow London stock brick or a warm orange-red variety, often decorated with carvings and white plaster mouldings.[224]


30 St Mary Axe, also known as "the Gherkin", towers over St Andrew Undershaft; modern styles juxtaposed by historic styles is seen often in London.
In the dense areas, most of the concentration is via medium- and high-rise buildings. London's skyscrapers, such as 30 St Mary Axe, Tower 42, the Broadgate Tower and One Canada Square, are mostly in the two financial districts, the City of London and Canary Wharf. High-rise development is restricted at certain sites if it would obstruct protected views of St Paul's Cathedral and other historic buildings. Nevertheless, there are a number of tall skyscrapers in central London (see Tall buildings in London), including the 95-storey Shard London Bridge, the tallest building in the United Kingdom.

Other notable modern buildings include City Hall in Southwark with its distinctive oval shape,[225] the Art Deco BBC Broadcasting House plus the Postmodernist British Library in Somers Town/Kings Cross and No 1 Poultry by James Stirling. What was formerly the Millennium Dome, by the Thames to the east of Canary Wharf, is now an entertainment venue called the O2 Arena.

Cityscape

The Houses of Parliament and Elizabeth Tower on the right foreground, the London Eye on the left foreground and The Shard with Canary Wharf in the background; seen in September 2014
Natural history
The London Natural History Society suggest that London is "one of the World's Greenest Cities" with more than 40 per cent green space or open water. They indicate that 2000 species of flowering plant have been found growing there and that the tidal Thames supports 120 species of fish.[226] They also state that over 60 species of bird nest in central London and that their members have recorded 47 species of butterfly, 1173 moths and more than 270 kinds of spider around London. London's wetland areas support nationally important populations of many water birds. London has 38 Sites of Special Scientific Interest (SSSIs), two national nature reserves and 76 local nature reserves.[227]

Amphibians are common in the capital, including smooth newts living by the Tate Modern, and common frogs, common toads, palmate newts and great crested newts. On the other hand, native reptiles such as slowworms, common lizards, grass snakes and adders, are mostly only seen in Outer London.[228]


A fox on Ayres Street, Southwark, South London
Among other inhabitants of London are 10,000 red foxes, so that there are now 16 foxes for every square mile (2.6 square kilometres) of London. These urban foxes are noticeably bolder than their country cousins, sharing the pavement with pedestrians and raising cubs in people's backyards. Foxes have even sneaked into the Houses of Parliament, where one was found asleep on a filing cabinet. Another broke into the grounds of Buckingham Palace, reportedly killing some of Queen Elizabeth II's prized pink flamingos. Generally, however, foxes and city folk appear to get along. A survey in 2001 by the London-based Mammal Society found that 80 per cent of 3,779 respondents who volunteered to keep a diary of garden mammal visits liked having them around. This sample cannot be taken to represent Londoners as a whole.[229][230]

Other mammals found in Greater London are hedgehogs, rats, mice, rabbit, shrew, vole, and squirrels.[231] In wilder areas of Outer London, such as Epping Forest, a wide variety of mammals are found, including hare, badger, field, bank and water vole, wood mouse, yellow-necked mouse, mole, shrew, and weasel, in addition to fox, squirrel and hedgehog. A dead otter was found at The Highway, in Wapping, about a mile from the Tower Bridge, which would suggest that they have begun to move back after being absent a hundred years from the city.[232] Ten of England's eighteen species of bats have been recorded in Epping Forest: soprano, nathusius and common pipistrelles, noctule, serotine, barbastelle, daubenton's, brown Long-eared, natterer's and leisler's.[233]

Among the strange sights seen in London have been a whale in the Thames,[234] while the BBC Two programme "Natural World: Unnatural History of London" shows pigeons using the London Underground to get around the city, a seal that takes fish from fishmongers outside Billingsgate Fish Market, and foxes that will "sit" if given sausages.[235]

Herds of red and fallow deer also roam freely within much of Richmond and Bushy Park. A cull takes place each November and February to ensure numbers can be sustained.[236] Epping Forest is also known for its fallow deer, which can frequently be seen in herds to the north of the Forest. A rare population of melanistic, black fallow deer is also maintained at the Deer Sanctuary near Theydon Bois. Muntjac deer, which escaped from deer parks at the turn of the twentieth century, are also found in the forest. While Londoners are accustomed to wildlife such as birds and foxes sharing the city, more recently urban deer have started becoming a regular feature, and whole herds of fallow deer come into residential areas at night to take advantage of London's green spaces.[237][238]


The 2011 census recorded that 2,998,264 people or 36.7% of London's population are foreign-born making London the city with the second largest immigrant population, behind New York City, in terms of absolute numbers. About 69% of children born in London in 2015 had at least one parent who was born abroad.[240] The table to the right shows the most common countries of birth of London residents. Note that some of the German-born population, in 18th position, are British citizens from birth born to parents serving in the British Armed Forces in Germany.[241]

With increasing industrialisation, London's population grew rapidly throughout the 19th and early 20th centuries, and it was for some time in the late 19th and early 20th centuries the most populous city in the world. Its population peaked at 8,615,245 in 1939 immediately before the outbreak of the Second World War, but had declined to 7,192,091 at the 2001 Census. However, the population then grew by just over a million between the 2001 and 2011 Censuses, to reach 8,173,941 in the latter enumeration.[242]

However, London's continuous urban area extends beyond the borders of Greater London and was home to 9,787,426 people in 2011,[57] while its wider metropolitan area has a population of between 12 and 14 million depending on the definition used.[243][244] According to Eurostat, London is the most populous city and metropolitan area of the European Union and the second most populous in Europe. During the period 1991–2001 a net 726,000 immigrants arrived in London.[245]

The region covers an area of 1,579 square kilometres (610 sq mi). The population density is 5,177 inhabitants per square kilometre (13,410/sq mi),[246] more than ten times that of any other British region.[247] In terms of population, London is the 19th largest city and the 18th largest metropolitan region.[248][249]

Age structure and median age
The children (aged younger than 14 years) constitutes a 21 percent in Outer London, and 28 percent in Inner London; the age group aged between 15 and 24 years are 12 percent in both Outer and Inner London; those aged between 25 and 44 years a 31 percent in Outer London and 40 percent in Inner London; the aged between 45 and 64 years formed the 26 and 21 percent in Outer and Inner London respectively; while in Outer London the aged 65 and older were the 13 percent, in Inner London were just the 9 percent.[250]

The median age of London in 2017 is 36.5 years old.[251]

Ethnic groups
Main article: Ethnic groups in London
According to the Office for National Statistics, based on the 2011 Census estimates, 59.8 per cent of the 8,173,941 inhabitants of London were White, with 44.9 per cent White British, 2.2 per cent White Irish, 0.1 per cent gypsy/Irish traveller and 12.1 per cent classified as Other White.[252]

20.9 per cent of Londoners are of Asian and mixed-Asian descent. 19.7 per cent are of full Asian descent, with those of mixed-Asian heritage comprising 1.2 of the population. Indians account for 6.6 per cent of the population, followed by Pakistanis and Bangladeshis at 2.7 per cent each. Chinese peoples account for 1.5 per cent of the population, with Arabs comprising 1.3 per cent. A further 4.9 per cent are classified as "Other Asian".[252]

15.6 per cent of London's population are of Black and mixed-Black descent. 13.3 per cent are of full Black descent, with those of mixed-Black heritage comprising 2.3 per cent. Black Africans account for 7.0 per cent of London's population, with 4.2 per cent as Black Caribbean and 2.1 per cent as "Other Black". 5.0 per cent are of mixed race.[252]

Across London, Black and Asian children outnumber White British children by about six to four in state schools.[253] Altogether at the 2011 census, of London's 1,624,768 population aged 0 to 15, 46.4 per cent were White, 19.8 per cent were Asian, 19 per cent were Black, 10.8 per cent were Mixed and 4 per cent represented another ethnic group.[254] In January 2005, a survey of London's ethnic and religious diversity claimed that there were more than 300 languages spoken in London and more than 50 non-indigenous communities with a population of more than 10,000.[255] Figures from the Office for National Statistics show that, in 2010, London's foreign-born population was 2,650,000 (33 per cent), up from 1,630,000 in 1997.

The 2011 census showed that 36.7 per cent of Greater London's population were born outside the UK.[256] A portion of the German-born population are likely to be British nationals born to parents serving in the British Armed Forces in Germany.[257] Estimates produced by the Office for National Statistics indicate that the five largest foreign-born groups living in London in the period July 2009 to June 2010 were those born in India, Poland, the Republic of Ireland, Bangladesh and Nigeria.[258]

According to the 2011 Census, the largest religious groupings are Christians (48.4 per cent), followed by those of no religion (20.7 per cent), Muslims (12.4 per cent), no response (8.5 per cent), Hindus (5.0 per cent), Jews (1.8 per cent), Sikhs (1.5 per cent), Buddhists (1.0 per cent) and other (0.6 per cent).

London has traditionally been Christian, and has a large number of churches, particularly in the City of London. The well-known St Paul's Cathedral in the City and Southwark Cathedral south of the river are Anglican administrative centres,[260] while the Archbishop of Canterbury, principal bishop of the Church of England and worldwide Anglican Communion, has his main residence at Lambeth Palace in the London Borough of Lambeth.[261]


St Paul's Cathedral, the seat of the Bishop of London

The BAPS Shri Swaminarayan Mandir London is the second-largest Hindu temple in England and Europe.
Important national and royal ceremonies are shared between St Paul's and Westminster Abbey.[262] The Abbey is not to be confused with nearby Westminster Cathedral, which is the largest Roman Catholic cathedral in England and Wales.[263] Despite the prevalence of Anglican churches, observance is very low within the Anglican denomination. Church attendance continues on a long, slow, steady decline, according to Church of England statistics.[264]

London is also home to sizeable Muslim, Hindu, Sikh, and Jewish communities.

Notable mosques include the East London Mosque in Tower Hamlets, which is allowed to give the Islamic call to prayer through loudspeakers, the London Central Mosque on the edge of Regent's Park[265] and the Baitul Futuh of the Ahmadiyya Muslim Community. Following the oil boom, increasing numbers of wealthy Middle-Eastern Arab Muslims have based themselves around Mayfair, Kensington, and Knightsbridge in West London.[266][267][268] There are large Bengali Muslim communities in the eastern boroughs of Tower Hamlets and Newham.[269]

Large Hindu communities are in the north-western boroughs of Harrow and Brent, the latter of which hosts what was, until 2006,[270] Europe's largest Hindu temple, Neasden Temple.[271] London is also home to 44 Hindu temples, including the BAPS Shri Swaminarayan Mandir London. There are Sikh communities in East and West London, particularly in Southall, home to one of the largest Sikh populations and the largest Sikh temple outside India.[272]

The majority of British Jews live in London, with significant Jewish communities in Stamford Hill, Stanmore, Golders Green, Finchley, Hampstead, Hendon and Edgware in North London. Bevis Marks Synagogue in the City of London is affiliated to London's historic Sephardic Jewish community. It is the only synagogue in Europe which has held regular services continuously for over 300 years. Stanmore and Canons Park Synagogue has the largest membership of any single Orthodox synagogue in the whole of Europe, overtaking Ilford synagogue (also in London) in 1998.[273] The community set up the London Jewish Forum in 2006 in response to the growing significance of devolved London Government.[274]

Accent
The accent of a 21st-century Londoner varies widely; what is becoming more and more common amongst the under-30s however is some fusion of Cockney with a whole array of ethnic accents, in particular Caribbean, which form an accent labelled Multicultural London English (MLE).[275] The other widely heard and spoken accent is RP (Received Pronunciation) in various forms, which can often be heard in the media and many of other traditional professions and beyond, although this accent is not limited to London and South East England, and can also be heard selectively throughout the whole UK amongst certain social groupings. Since the turn of the century the Cockney dialect is less common in the East End and has 'migrated' east to Havering and the county of Essex.[276][277]

Economy
Main article: Economy of London

The City of London, one of the largest financial centres in the world[278]
London's gross regional product in 2018 was almost £500 billion, around a quarter of UK GDP.[279] London has five major business districts: the City, Westminster, Canary Wharf, Camden & Islington and Lambeth & Southwark. One way to get an idea of their relative importance is to look at relative amounts of office space: Greater London had 27 million m2 of office space in 2001, and the City contains the most space, with 8 million m2 of office space. London has some of the highest real estate prices in the world.[280][281] London is the world's most expensive office market for the last three years according to world property journal (2015) report.[282] As of 2015 the residential property in London is worth $2.2 trillion – same value as that of Brazil's annual GDP.[283] The city has the highest property prices of any European city according to the Office for National Statistics and the European Office of Statistics.[284] On average the price per square metre in central London is €24,252 (April 2014). This is higher than the property prices in other G8 European capital cities; Berlin €3,306, Rome €6,188 and Paris €11,229.[285]

The City of London

The London Stock Exchange at Paternoster Square and Temple Bar
London's finance industry is based in the City of London and Canary Wharf, the two major business districts in London. London is one of the pre-eminent financial centres of the world as the most important location for international finance.[286][287] London took over as a major financial centre shortly after 1795 when the Dutch Republic collapsed before the Napoleonic armies. For many bankers established in Amsterdam (e.g. Hope, Baring), this was only time to move to London. The London financial elite was strengthened by a strong Jewish community from all over Europe capable of mastering the most sophisticated financial tools of the time.[112] This unique concentration of talents accelerated the transition from the Commercial Revolution to the Industrial Revolution. By the end of the 19th century, Britain was the wealthiest of all nations, and London a leading financial centre. Still, as of 2016 London tops the world rankings on the Global Financial Centres Index (GFCI),[288] and it ranked second in A.T. Kearney's 2018 Global Cities Index.[289]


A view from Westminster Millennium Pier on the River Thames, December 2018
London's largest industry is finance, and its financial exports make it a large contributor to the UK's balance of payments. Around 325,000 people were employed in financial services in London until mid-2007. London has over 480 overseas banks, more than any other city in the world. It is also the world's biggest currency trading centre, accounting for some 37 per cent of the $5.1 trillion average daily volume, according to the BIS.[290] Over 85 per cent (3.2 million) of the employed population of greater London works in the services industries. Because of its prominent global role, London's economy had been affected by the financial crisis of 2007–2008. However, by 2010 the City has recovered; put in place new regulatory powers, proceeded to regain lost ground and re-established London's economic dominance.[291] Along with professional services headquarters, the City of London is home to the Bank of England, London Stock Exchange, and Lloyd's of London insurance market.

Over half of the UK's top 100 listed companies (the FTSE 100) and over 100 of Europe's 500 largest companies have their headquarters in central London. Over 70 per cent of the FTSE 100 are within London's metropolitan area, and 75 per cent of Fortune 500 companies have offices in London.[292]

Media and technology
Media companies are concentrated in London and the media distribution industry is London's second most competitive sector.[293] The BBC is a significant employer, while other broadcasters also have headquarters around the City. Many national newspapers are edited in London. London is a major retail centre and in 2010 had the highest non-food retail sales of any city in the world, with a total spend of around £64.2 billion.[294] The Port of London is the second-largest in the United Kingdom, handling 45 million tonnes of cargo each year.[295]

A growing number of technology companies are based in London notably in East London Tech City, also known as Silicon Roundabout. In April 2014, the city was among the first to receive a geoTLD.[296] In February 2014 London was ranked as the European City of the Future[297] in the 2014/15 list by FDi Magazine.[298]

The gas and electricity distribution networks that manage and operate the towers, cables and pressure systems that deliver energy to consumers across the city are managed by National Grid plc, SGN[299] and UK Power Networks.[300]


Transport
Main articles: Transport in London and Infrastructure in London

Journeys in Greater London by mode from 1997–2017[309]

Black London taxis
Transport is one of the four main areas of policy administered by the Mayor of London,[310] however the mayor's financial control does not extend to the longer distance rail network that enters London. In 2007 he assumed responsibility for some local lines, which now form the London Overground network, adding to the existing responsibility for the London Underground, trams and buses. The public transport network is administered by Transport for London (TfL).

The lines that formed the London Underground, as well as trams and buses, became part of an integrated transport system in 1933 when the London Passenger Transport Board or London Transport was created. Transport for London is now the statutory corporation responsible for most aspects of the transport system in Greater London, and is run by a board and a commissioner appointed by the Mayor of London.[311]

Aviation
Main article: Airports of London

London Heathrow Airport is the busiest airport in Europe as well as the second busiest in the world for international passenger traffic. (Terminal 5C is pictured)
London is a major international air transport hub with the busiest city airspace in the world. Eight airports use the word London in their name, but most traffic passes through six of these. Additionally, various other airports also serve London, catering primarily to general aviation flights.

London Heathrow Airport, in Hillingdon, West London, was for many years the busiest airport in the world for international traffic, and is the major hub of the nation's flag carrier, British Airways.[312] In March 2008 its fifth terminal was opened.[313] In 2014, Dubai gained from Heathrow the leading position in terms of international passenger traffic.[314]
London Gatwick Airport,[315] south of London in West Sussex, handles flights to more destinations than any other UK airport[316] and is the main base of easyJet,[317] the UK's largest airline by number of passengers.[318]
London Stansted Airport,[319] north-east of London in Essex, has flights that serve the greatest number of European destinations of any UK airport[320] and is the main base of Ryanair,[321] the world's largest international airline by number of international passengers.[322]
London Luton Airport, to the north of London in Bedfordshire, is used by several budget airlines for short-haul flights.[323]
London City Airport, the most central airport and the one with the shortest runway, in Newham, East London, is focused on business travellers, with a mixture of full-service short-haul scheduled flights and considerable business jet traffic.[324]
London Southend Airport, east of London in Essex, is a smaller, regional airport that caters for short-haul flights on a limited, though growing, number of airlines.[325] In 2017, international passengers made up over 95% of the total at Southend, the highest proportion of any London airport.[326]
Rail
Underground and DLR

The London Underground is the world's oldest and third-longest rapid transit system.
The London Underground, commonly referred to as the Tube, is the oldest[327] and third longest[328] metro system in the world. The system serves 270 stations[329] and was formed from several private companies, including the world's first underground electric line, the City and South London Railway.[330] It dates from 1863.[331]

Over four million journeys are made every day on the Underground network, over 1 billion each year.[332] An investment programme is attempting to reduce congestion and improve reliability, including £6.5 billion (€7.7 billion) spent before the 2012 Summer Olympics.[333] The Docklands Light Railway (DLR), which opened in 1987, is a second, more local metro system using smaller and lighter tram-type vehicles that serve the Docklands, Greenwich and Lewisham.

Suburban
There are more than 360 railway stations in the London Travelcard Zones on an extensive above-ground suburban railway network. South London, particularly, has a high concentration of railways as it has fewer Underground lines. Most rail lines terminate around the centre of London, running into eighteen terminal stations, with the exception of the Thameslink trains connecting Bedford in the north and Brighton in the south via Luton and Gatwick airports.[334] London has Britain's busiest station by number of passengers – Waterloo, with over 184 million people using the interchange station complex (which includes Waterloo East station) each year.[335][336] Clapham Junction is the busiest station in Europe by the number of trains passing.

With the need for more rail capacity in London, Crossrail is expected to open in 2020/21.[337] It will be a new railway line running east to west through London and into the Home Counties with a branch to Heathrow Airport.[338] It is Europe's biggest construction project, with a £15 billion projected cost.[339][340]

Inter-city and international

St Pancras International is the main terminal for high speed Eurostar and High Speed 1 services, as well as commuter suburban Thameslink and inter-city East Midlands Railway services.
London is the centre of the National Rail network, with 70 per cent of rail journeys starting or ending in London.[341] Like suburban rail services, regional and inter-city trains depart from several termini around the city centre, linking London with the rest of Britain including Birmingham, Brighton, Reading, Bristol, Cardiff, Chester, Holyhead (for Dublin), Derby, Nottingham, Exeter, Sheffield, York, Southampton, Leeds, Liverpool, Manchester, Cambridge, Newcastle upon Tyne, Edinburgh and Glasgow.

Some international railway services to Continental Europe were operated during the 20th century as boat trains, such as the Admiraal de Ruijter to Amsterdam and the Night Ferry to Paris and Brussels. The opening of the Channel Tunnel in 1994 connected London directly to the continental rail network, allowing Eurostar services to begin. Since 2007, high-speed trains link St. Pancras International with Lille, Calais, Paris, Disneyland Paris, Brussels, Amsterdam and other European tourist destinations via the High Speed 1 rail link and the Channel Tunnel.[342] The first high-speed domestic trains started in June 2009 linking Kent to London.[343] There are plans for a second high speed line linking London to the Midlands, North West England, and Yorkshire.

Freight
Although rail freight levels are far down compared to their height, significant quantities of cargo are also carried into and out of London by rail; chiefly building materials and landfill waste.[344] As a major hub of the British railway network, London's tracks also carry large amounts of freight for the other regions, such as container freight from the Channel Tunnel and English Channel ports, and nuclear waste for reprocessing at Sellafield.[344]

Buses and trams

The red double decker bus is an iconic symbol of London.
London's bus network is one of the largest in the world, running 24 hours a day, with about 8,500 buses, more than 700 bus routes and around 19,500 bus stops.[345] In 2013, the network had more than 2 billion commuter trips per annum, more than the Underground.[345] Around £850 million is taken in revenue each year. London has the largest wheelchair accessible network in the world[346] and, from the 3rd quarter of 2007, became more accessible to hearing and visually impaired passengers as audio-visual announcements were introduced. The distinctive red double-decker buses are an internationally recognised trademark of London transport along with black cabs and the Tube.[347][348]

London has a modern tram network, known as Tramlink, centred on Croydon in South London. The network has 39 stops and four routes, and carried 28 million people in 2013.[349] Since June 2008 Transport for London has completely owned Tramlink, and it plans to spend £54m by 2015 on maintenance, renewals, upgrades and capacity enhancements.[350]

Cable car
London's first and only cable car, known as the Emirates Air Line, opened in June 2012. Crossing the River Thames, linking Greenwich Peninsula and the Royal Docks in the east of the city, the cable car is integrated with London's Oyster Card ticketing system, although special fares are charged. Costing £60 million to build, it carries over 3,500 passengers every day, although this is very much lower than its capacity. Similar to the Santander Cycles bike hire scheme, the cable car is sponsored in a 10-year deal by the airline Emirates.

Cycling
Main article: Cycling in London

Santander Cycle Hire near Victoria in Central London
In the whole Greater London Area, around 650,000 people use a bike everyday.[351] But out of a total population of around 8.8 million,[352] this means that just around 7% of Greater London's population use a bike on an average day. This is a small proportion, when compared to many other cities in the world.[353] A reason may well be the poor investments for cycling in London of about £110 million per year,[354] equating to around £12 per person, which can be compared to £22 in the Netherlands.[355]

Cycling is nevertheless becoming increasingly popular way to get around London. The launch of a cycle hire scheme in July 2010 has been successful and generally well received. The London Cycling Campaign lobbies for better provision.[356] There are many cycle routes in London, including several Cycle Superhighways.

Port and river boats
From being the largest port in the world, the Port of London is now only the second-largest in the United Kingdom, handling 45 million tonnes of cargo each year.[295] Most of this actually passes through the Port of Tilbury, outside the boundary of Greater London.

London has frequent river boat services on the Thames known as Thames Clippers. These run up to every 20 minutes between Embankment Pier and North Greenwich Pier. The Woolwich Ferry, with 2.5 million passengers every year,[357] is a frequent service linking the North and South Circular Roads. Other operators run both commuter and tourist boat services in London.

Roads
Although the majority of journeys involving central London are made by public transport, car travel is common in the suburbs. The inner ring road (around the city centre), the North and South Circular roads (in the suburbs), and the outer orbital motorway (the M25, outside the built-up area) encircle the city and are intersected by a number of busy radial routes—but very few motorways penetrate into inner London. A plan for a comprehensive network of motorways throughout the city (the Ringways Plan) was prepared in the 1960s but was mostly cancelled in the early 1970s. The M25 is the second-longest ring-road motorway in Europe at 117 mi (188 km) long.[358] The A1 and M1 connect London to Leeds, and Newcastle and Edinburgh.

London is notorious for its traffic congestion, the average speed of a car in the rush hour being 10.6 mph (17.1 km/h).[359]

In 2003, a congestion charge was introduced to reduce traffic volumes in the city centre. With a few exceptions, motorists are required to pay £10 per day to drive within a defined zone encompassing much of central London.[360][361] Motorists who are residents of the defined zone can buy a greatly reduced season pass.[362] London government initially expected the Congestion Charge Zone to increase daily peak period Underground and bus users by 20,000 people, reduce road traffic by 10 to 15 per cent, increase traffic speeds by 10 to 15 per cent, and reduce queues by 20 to 30 per cent.[363] Over the course of several years, the average number of cars entering the centre of London on a weekday was reduced from 195,000 to 125,000 cars – a 35-per-cent reduction of vehicles driven per day.[364]

Education
Main article: Education in London
Tertiary education

Imperial College London, a technical research university in South Kensington
London is a major global centre of higher education teaching and research and has the largest concentration of higher education institutes in Europe.[49] According to the QS World University Rankings 2015/16, London has the greatest concentration of top class universities in the world[365][366] and its international student population of around 110,000 is larger than any other city in the world.[367] A 2014 PricewaterhouseCoopers report termed London the global capital of higher education.[368]


King's College London, established by Royal Charter in 1829, is one of the founding colleges of the University of London
A number of world-leading education institutions are based in London. In the 2014/15 QS World University Rankings, Imperial College London is ranked joint 2nd in the world, University College London (UCL) is ranked 5th, and King's College London (KCL) is ranked 16th.[369] The London School of Economics has been described as the world's leading social science institution for both teaching and research.[370] The London Business School is considered one of the world's leading business schools and in 2015 its MBA programme was ranked second best in the world by the Financial Times.[371]

With 120,000 students in London,[372] the federal University of London is the largest contact teaching university in the UK.[373] It includes five multi-faculty universities – City, King's College London, Queen Mary, Royal Holloway and UCL – and a number of smaller and more specialised institutions including Birkbeck, the Courtauld Institute of Art, Goldsmiths, Guildhall School of Music and Drama, the London Business School, the London School of Economics, the London School of Hygiene & Tropical Medicine, the Royal Academy of Music, the Central School of Speech and Drama, the Royal Veterinary College and the School of Oriental and African Studies.[374] Members of the University of London have their own admissions procedures, and some award their own degrees.

A number of universities in London are outside the University of London system, including Brunel University, Imperial College London, Kingston University, London Metropolitan University,[375] University of East London, University of West London, University of Westminster, London South Bank University, Middlesex University, and University of the Arts London (the largest university of art, design, fashion, communication and the performing arts in Europe).[376] In addition there are three international universities in London – Regent's University London, Richmond, The American International University in London and Schiller International University.


The front façade of the Royal College of Music
London is home to five major medical schools – Barts and The London School of Medicine and Dentistry (part of Queen Mary), King's College London School of Medicine (the largest medical school in Europe), Imperial College School of Medicine, UCL Medical School and St George's, University of London – and has many affiliated teaching hospitals. It is also a major centre for biomedical research, and three of the UK's eight academic health science centres are based in the city – Imperial College Healthcare, King's Health Partners and UCL Partners (the largest such centre in Europe).[377] Additionally, many biomedical and biotechnology spin out companies from these research institutions are based around the city, most prominently in White City.

There are a number of business schools in London, including the London School of Business and Finance, Cass Business School (part of City University London), Hult International Business School, ESCP Europe, European Business School London, Imperial College Business School, the London Business School and the UCL School of Management. London is also home to many specialist arts education institutions, including the Academy of Live and Recorded Arts, Central School of Ballet, LAMDA, London College of Contemporary Arts (LCCA), London Contemporary Dance School, National Centre for Circus Arts, RADA, Rambert School of Ballet and Contemporary Dance, the Royal College of Art, the Royal College of Music and Trinity Laban.

Primary and secondary education
The majority of primary and secondary schools and further-education colleges in London are controlled by the London boroughs or otherwise state-funded; leading examples include Ashbourne College, Bethnal Green Academy, Brampton Manor Academy, City and Islington College, City of Westminster College, David Game College, Ealing, Hammersmith and West London College, Leyton Sixth Form College, London Academy of Excellence, Tower Hamlets College, and Newham Collegiate Sixth Form Centre. There are also a number of private schools and colleges in London, some old and famous, such as City of London School, Harrow, St Paul's School, Haberdashers' Aske's Boys' School, University College School, The John Lyon School, Highgate School and Westminster School.

Culture
Main article: Culture of London
Leisure and entertainment
See also: List of annual events in London and West End theatre

Piccadilly Circus
Leisure is a major part of the London economy, with a 2003 report attributing a quarter of the entire UK leisure economy to London[378] at 25.6 events per 1000 people.[379] Globally, the city is amongst the big four fashion capitals of the world, and according to official statistics, London is the world's third busiest film production centre, presents more live comedy than any other city,[380] and has the biggest theatre audience of any city in the world.[381]


Harrods in Knightsbridge
Within the City of Westminster in London, the entertainment district of the West End has its focus around Leicester Square, where London and world film premieres are held, and Piccadilly Circus, with its giant electronic advertisements.[382] London's theatre district is here, as are many cinemas, bars, clubs, and restaurants, including the city's Chinatown district (in Soho), and just to the east is Covent Garden, an area housing speciality shops. The city is the home of Andrew Lloyd Webber, whose musicals have dominated the West End theatre since the late 20th century.[383] The United Kingdom's Royal Ballet, English National Ballet, Royal Opera, and English National Opera are based in London and perform at the Royal Opera House, the London Coliseum, Sadler's Wells Theatre, and the Royal Albert Hall, as well as touring the country.[384]


Scene of the annual Notting Hill Carnival, 2014
Islington's 1 mile (1.6 km) long Upper Street, extending northwards from Angel, has more bars and restaurants than any other street in the United Kingdom.[385] Europe's busiest shopping area is Oxford Street, a shopping street nearly 1 mile (1.6 km) long, making it the longest shopping street in the UK. Oxford Street is home to vast numbers of retailers and department stores, including the world-famous Selfridges flagship store.[386] Knightsbridge, home to the equally renowned Harrods department store, lies to the south-west.

London is home to designers Vivienne Westwood, Galliano, Stella McCartney, Manolo Blahnik, and Jimmy Choo, among others; its renowned art and fashion schools make it an international centre of fashion alongside Paris, Milan, and New York City. London offers a great variety of cuisine as a result of its ethnically diverse population. Gastronomic centres include the Bangladeshi restaurants of Brick Lane and the Chinese restaurants of Chinatown.[387]


Shakespeare's Globe is a modern reconstruction of the Globe Theatre on the south bank of the River Thames.
There is a variety of annual events, beginning with the relatively new New Year's Day Parade, a fireworks display at the London Eye; the world's second largest street party, the Notting Hill Carnival, is held on the late August Bank Holiday each year. Traditional parades include November's Lord Mayor's Show, a centuries-old event celebrating the annual appointment of a new Lord Mayor of the City of London with a procession along the streets of the City, and June's Trooping the Colour, a formal military pageant performed by regiments of the Commonwealth and British armies to celebrate the Queen's Official Birthday.[388] The Boishakhi Mela is a Bengali New Year festival celebrated by the British Bangladeshi community. It is the largest open-air Asian festival in Europe. After the Notting Hill Carnival, it is the second-largest street festival in the United Kingdom attracting over 80,000 visitors from across the country.[389]

Literature, film and television
Main articles: London in fiction, London in film, List of television shows set in London, and London Television Archive

Sherlock Holmes Museum in Baker Street, bearing the number 221B
London has been the setting for many works of literature. The pilgrims in Geoffrey Chaucer's late 14th-century Canterbury Tales set out for Canterbury from London – specifically, from the Tabard inn, Southwark. William Shakespeare spent a large part of his life living and working in London; his contemporary Ben Jonson was also based there, and some of his work, most notably his play The Alchemist, was set in the city.[390] A Journal of the Plague Year (1722) by Daniel Defoe is a fictionalisation of the events of the 1665 Great Plague.[390]

The literary centres of London have traditionally been hilly Hampstead and (since the early 20th century) Bloomsbury. Writers closely associated with the city are the diarist Samuel Pepys, noted for his eyewitness account of the Great Fire, Charles Dickens, whose representation of a foggy, snowy, grimy London of street sweepers and pickpockets has been a major influence on people's vision of early Victorian London, and Virginia Woolf, regarded as one of the foremost modernist literary figures of the 20th century.[390] Later important depictions of London from the 19th and early 20th centuries are Dickens' novels, and Arthur Conan Doyle's Sherlock Holmes stories.[390] Also of significance is Letitia Elizabeth Landon's Calendar of the London Seasons (1834). Modern writers pervasively influenced by the city include Peter Ackroyd, author of a "biography" of London, and Iain Sinclair, who writes in the genre of psychogeography.

	Wikisource has original text related to this article:
'Calendar of the London Seasons', by L. E. L.

Keats House, where Keats wrote his Ode to a Nightingale. The village of Hampstead has historically been a literary centre in London.
London has played a significant role in the film industry. Major studios within or bordering London include Twickenham, Ealing, Shepperton, Pinewood, Elstree and Borehamwood,[391] and a special effects and post-production community centred in Soho. Working Title Films has its headquarters in London.[392] London has been the setting for films including Oliver Twist (1948), Scrooge (1951), Peter Pan (1953), The 101 Dalmatians (1961), My Fair Lady (1964), Mary Poppins (1964), Blowup (1966), The Long Good Friday (1980), Notting Hill (1999), Love Actually (2003), V For Vendetta (2005), Sweeney Todd: The Demon Barber of Fleet Street (2008) and The King's Speech (2010). Notable actors and filmmakers from London include; Charlie Chaplin, Alfred Hitchcock, Michael Caine, Helen Mirren, Gary Oldman, Christopher Nolan, Jude Law, Benedict Cumberbatch, Tom Hardy, Keira Knightley and Daniel Day-Lewis. As of 2008, the British Academy Film Awards have taken place at the Royal Opera House. London is a major centre for television production, with studios including BBC Television Centre, The Fountain Studios and The London Studios. Many television programmes have been set in London, including the popular television soap opera EastEnders, broadcast by the BBC since 1985.

Museums, art galleries and libraries

Aerial view of Albertopolis. Albert Memorial, Royal Albert Hall and Royal College of Art are visible near the top; Victoria and Albert Museum and Natural History Museum at the lower end; Imperial College, Royal College of Music, and Science Museum lying in between.
London is home to many museums, galleries, and other institutions, many of which are free of admission charges and are major tourist attractions as well as playing a research role. The first of these to be established was the British Museum in Bloomsbury, in 1753. Originally containing antiquities, natural history specimens, and the national library, the museum now has 7 million artefacts from around the globe. In 1824, the National Gallery was founded to house the British national collection of Western paintings; this now occupies a prominent position in Trafalgar Square.

The British Library is one of the largest libraries in the world, and the national library of the United Kingdom.[393] The are many other research libraries, including the Wellcome Library and Dana Centre, as well as university libraries, including the British Library of Political and Economic Science at LSE, the Central Library at Imperial, the Maughan Library at King's, and the Senate House Libraries at the University of London.[394][395]

In the latter half of the 19th century the locale of South Kensington was developed as "Albertopolis", a cultural and scientific quarter. Three major national museums are there: the Victoria and Albert Museum (for the applied arts), the Natural History Museum, and the Science Museum. The National Portrait Gallery was founded in 1856 to house depictions of figures from British history; its holdings now comprise the world's most extensive collection of portraits.[396] The national gallery of British art is at Tate Britain, originally established as an annexe of the National Gallery in 1897. The Tate Gallery, as it was formerly known, also became a major centre for modern art; in 2000, this collection moved to Tate Modern, a new gallery housed in the former Bankside Power Station.

Music

The Royal Albert Hall hosts concerts and musical events.
London is one of the major classical and popular music capitals of the world and hosts major music corporations, such as Universal Music Group International and Warner Music Group, as well as countless bands, musicians and industry professionals. The city is also home to many orchestras and concert halls, such as the Barbican Arts Centre (principal base of the London Symphony Orchestra and the London Symphony Chorus), Cadogan Hall (Royal Philharmonic Orchestra) and the Royal Albert Hall (The Proms).[384] London's two main opera houses are the Royal Opera House and the London Coliseum.[384] The UK's largest pipe organ is at the Royal Albert Hall. Other significant instruments are at the cathedrals and major churches. Several conservatoires are within the city: Royal Academy of Music, Royal College of Music, Guildhall School of Music and Drama and Trinity Laban.


Abbey Road Studios, 3 Abbey Road, St John's Wood, City of Westminster
London has numerous venues for rock and pop concerts, including the world's busiest indoor venue, The O2 Arena[397] and Wembley Arena, as well as many mid-sized venues, such as Brixton Academy, the Hammersmith Apollo and the Shepherd's Bush Empire.[384] Several music festivals, including the Wireless Festival, South West Four, Lovebox, and Hyde Park's British Summer Time are all held in London.[398] The city is home to the original Hard Rock Cafe and the Abbey Road Studios, where The Beatles recorded many of their hits. In the 1960s, 1970s and 1980s, musicians and groups like Elton John, Pink Floyd, Cliff Richard, David Bowie, Queen, The Kinks, The Rolling Stones, The Who, Eric Clapton, Led Zeppelin, The Small Faces, Iron Maiden, Fleetwood Mac, Elvis Costello, Cat Stevens, The Police, The Cure, Madness, The Jam, Ultravox, Spandau Ballet, Culture Club, Dusty Springfield, Phil Collins, Rod Stewart, Adam Ant, Status Quo and Sade, derived their sound from the streets and rhythms of London.[399]

London was instrumental in the development of punk music,[400] with figures such as the Sex Pistols, The Clash,[399] and Vivienne Westwood all based in the city. More recent artists to emerge from the London music scene include George Michael's Wham!, Kate Bush, Seal, the Pet Shop Boys, Bananarama, Siouxsie and the Banshees, Bush, the Spice Girls, Jamiroquai, Blur, McFly, The Prodigy, Gorillaz, Bloc Party, Mumford & Sons, Coldplay, Amy Winehouse, Adele, Sam Smith, Ed Sheeran, Paloma Faith, Ellie Goulding, One Direction and Florence and the Machine.[401][402][403] London is also a centre for urban music. In particular the genres UK garage, drum and bass, dubstep and grime evolved in the city from the foreign genres of hip hop and reggae, alongside local drum and bass. Music station BBC Radio 1Xtra was set up to support the rise of local urban contemporary music both in London and in the rest of the United Kingdom.

Notable people
Main article: List of people from London
Recreation
Parks and open spaces
Main articles: Parks and open spaces in London and Royal Parks of London
See also: List of Sites of Special Scientific Interest in Greater London and List of local nature reserves in Greater London
Hyde Park
Aerial view of Hyde Park
Lake with London Eye in the background
St. James's Park lake with the London Eye in the distance
A 2013 report by the City of London Corporation said that London is the "greenest city" in Europe with 35,000 acres of public parks, woodlands and gardens.[404] The largest parks in the central area of London are three of the eight Royal Parks, namely Hyde Park and its neighbour Kensington Gardens in the west, and Regent's Park to the north.[405] Hyde Park in particular is popular for sports and sometimes hosts open-air concerts. Regent's Park contains London Zoo, the world's oldest scientific zoo, and is near Madame Tussauds Wax Museum.[406][407] Primrose Hill, immediately to the north of Regent's Park, at 256 feet (78 m)[408] is a popular spot from which to view the city skyline.

Close to Hyde Park are smaller Royal Parks, Green Park and St. James's Park.[409] A number of large parks lie outside the city centre, including Hampstead Heath and the remaining Royal Parks of Greenwich Park to the southeast[410] and Bushy Park and Richmond Park (the largest) to the southwest,[411][412] Hampton Court Park is also a royal park, but, because it contains a palace, it is administered by the Historic Royal Palaces, unlike the eight Royal Parks.[413]

Close to Richmond Park is Kew Gardens which has the world's largest collection of living plants. In 2003, the gardens were put on the UNESCO list of World Heritage Sites.[414] There are also parks administered by London's borough Councils, including Victoria Park in the East End and Battersea Park in the centre. Some more informal, semi-natural open spaces also exist, including the 320-hectare (790-acre) Hampstead Heath of North London,[415] and Epping Forest, which covers 2,476 hectares (6,118 acres)[416] in the east. Both are controlled by the City of London Corporation.[417][418] Hampstead Heath incorporates Kenwood House, a former stately home and a popular location in the summer months when classical musical concerts are held by the lake, attracting thousands of people every weekend to enjoy the music, scenery and fireworks.[419]

Epping Forest is a popular venue for various outdoor activities, including mountain biking, walking, horse riding, golf, angling, and orienteering.[420]

Walking

The River Wandle, Carshalton, in the London Borough of Sutton
Walking is a popular recreational activity in London. Areas that provide for walks include Wimbledon Common, Epping Forest, Hampton Court Park, Hampstead Heath, the eight Royal Parks, canals and disused railway tracks.[421] Access to canals and rivers has improved recently, including the creation of the Thames Path, some 28 miles (45 km) of which is within Greater London, and The Wandle Trail; this runs 12 miles (19 km) through South London along the River Wandle, a tributary of the River Thames.[422]

Other long distance paths, linking green spaces, have also been created, including the Capital Ring, the Green Chain Walk, London Outer Orbital Path ("Loop"), Jubilee Walkway, Lea Valley Walk, and the Diana, Princess of Wales Memorial Walk.[423]

Sport
Main article: Sport in London

Wembley Stadium, home of the England football team, has a 90,000 capacity. It is the UK's biggest stadium.
London has hosted the Summer Olympics three times: in 1908, 1948, and 2012,[424][425] making it the first city to host the modern Games three times.[53] The city was also the host of the British Empire Games in 1934.[426] In 2017, London hosted the World Championships in Athletics for the first time.[427]

London's most popular sport is football and it has five clubs in the English Premier League as of the 2019–20 season: Arsenal, Chelsea, Crystal Palace, Tottenham Hotspur, and West Ham United.[428] Other professional teams in London are Fulham, Queens Park Rangers, Brentford, Millwall, Charlton Athletic, AFC Wimbledon and Barnet.


Twickenham, home of the England rugby union team, has an 82,000 capacity, the world's largest rugby union stadium.
From 1924, the original Wembley Stadium was the home of the English national football team. It hosted the 1966 FIFA World Cup Final, with England defeating West Germany, and served as the venue for the FA Cup Final as well as rugby league's Challenge Cup final.[429] The new Wembley Stadium serves exactly the same purposes and has a capacity of 90,000.[430]

Two Aviva Premiership rugby union teams are based in London, Saracens and Harlequins.[431] London Scottish, London Welsh and London Irish play in the RFU Championship club and other rugby union clubs in the city include Richmond F.C., Rosslyn Park F.C., Westcombe Park R.F.C. and Blackheath F.C.. Twickenham Stadium in south-west London hosts home matches for the England national rugby union team and has a capacity of 82,000 now that the new south stand has been completed.[432]

'''

In [0]:
# Parse the document with spaCy
doc = nlp(text)

# Extract semi-structured statements
statements = textacy.extract.semistructured_statements(doc, "London")

for statement in statements:
    subject, verb, fact = statement
    print(f" - {fact}")

 - a major settlement for two millennia
 - the world's most populous city from c. 1831 to 1925.[59]


 - beyond all comparison the largest town in England
 - also a centre of England's Jewish population before their expulsion by Edward I in 1290
 - still very compact
 - notorious.[122]


 - the seat of the Government of the United Kingdom
 - vulnerable to flooding.[183
 - "one of the World's Greenest Cities" with more than 40 per cent green space or open water
 - the most populous city and metropolitan area of the European Union and the second most populous in Europe
 - the 19th largest city and the 18th largest metropolitan region.[248][249]

Age structure and median age

 - Christian, and has a large number of churches, particularly in the City of London
 - the second-largest Hindu temple in England and Europe.

 - also home to sizeable Muslim, Hindu, Sikh, and Jewish communities.


 - also home to 44 Hindu temples, including the BAPS Shri Swaminarayan Mandir London
 - the world's mo

**Now let's do for Nepal's wikipedia article.**

In [0]:
text='''

Nepal(Nepali: नेपाल[neˈpal]), officially the Federal Democratic Republic of Nepal, [12] is a landlocked country in South Asia.It is located mainly in the Himalayas, but also includes parts of the Indo - Gangetic Plain.With an estimated population of 26.4 million, it is 48 th largest country by population and 93 rd largest country by area. [8][13] It borders China in the north and India in the south, east and west
while Bangladesh is located within only 27 km(17 mi) of its southeastern tip and Bhutan is separated from it by the Indian state of Sikkim.Nepal has a diverse geography, including fertile plains, [14] subalpine forested hills, and eight of the world 's ten tallest mountains, including Mount Everest, the highest point on Earth. Kathmandu is the capital and the largest city. Nepal is a multiethnic country with Nepali as the official language.

The name "Nepal"
is first recorded in texts from the Vedic period of the Indian subcontinent, the era in ancient India when Hinduism was founded, the predominant religion of the country.In the middle of the first millennium BCE, Gautama Buddha, the founder of Buddhism, was born in Lumbini in southern Nepal.Parts of northern Nepal were intertwined with the culture of Tibet.The centrally located Kathmandu Valley is intertwined with the culture of Indo - Aryans, [15] and was the seat of the prosperous Newar confederacy known as Nepal Mandala.The Himalayan branch of the ancient Silk Road was dominated by the valley 's traders. The cosmopolitan region developed distinct traditional art and architecture. By the 18th century, the Gorkha Kingdom achieved the unification of Nepal. The Shah dynasty established the Kingdom of Nepal and later formed an alliance with the British Empire, under its Rana dynasty of premiers. The country was never colonized but served as a buffer state between Imperial China and British India.[16][17][18] Parliamentary democracy was introduced in 1951, but was twice suspended by Nepalese monarchs, in 1960 and 2005. The Nepalese Civil War in the 1990s and early 2000s resulted in the establishment of a secular republic in 2008, ending the world'
s last Hindu monarchy. [19]

The Constitution of Nepal, adopted in 2015, affirms Nepal as a secular federal parliamentary republic divided into seven provinces. [20] Nepal was admitted to the United Nations in 1955, and friendship treaties were signed with India in 1950 and the People 's Republic of China in 1960.[21][22] Nepal hosts the permanent secretariat of the South Asian Association for Regional Cooperation (SAARC), of which it is a founding member. Nepal is also a member of the Non-Aligned Movement and the Bay of Bengal Initiative. The military of Nepal is the fifth largest in South Asia; it is notable for its Gurkha history, particularly during the world wars, and has been a significant contributor to United Nations peacekeeping operations.

Before the unification of Nepal, the Kathmandu valley was known as Nepal Rajya. [a] The precise origin of the term Nepāl is uncertain.A number of plausible theories are found in religious as well as academic texts.Nepal appears in ancient Indian literary texts dated as far back as the fourth century BC.However, an absolute chronology can not be established, as even the oldest texts may contain anonymous contributions dating as late as the early modern period.On the other hand, academic attempts to provide a plausible theory suffer from lack of a complete picture of history, and insufficient understanding of linguistics or of relevant Indo - European and Tibeto - Burman languages. [24]

According to Hindu mythology, Nepal derives its name from an ancient Hindu sage called Ne, referred to variously as Ne Muni or Nemi.According to Pashupati Purana, as a place protected by Ne, the country in the heart of the Himalayas came to be known as Nepal. [b] According to Nepal Mahatmya, [c] Nemi was charged with protection of the country by Pashupati. [25] According to Buddhist mythology, Manjushri Bodhisattva drained a primordial lake of serpents to create the Nepal valley and proclaimed that Adi - Buddha Ne would take care of the community that would settle it.As the cherished of Ne, the valley would be called Nepal. [26] According to Gopalarajvamshavali, the genealogy of ancient Gopala dynasty compiled circa 1380 s, Nepal is named after Nepa the cowherd, the founder of the Nepali scion of the Abhiras.In this account, the cow that issued milk to the spot, at which Nepa discovered the Jyotirlinga of Pashupatinath upon investigation, was also named Ne. [24]

Norwegian Indologist Christian Lassen proposed that Nepala was a compound of Nipa(foot of a mountain) and - ala(short suffix
  for alaya which means abode), and therefore, Nepala meant "abode at the foot of the mountain".He considered Ne Muni to be a fabrication. [27] Indologist Sylvain Levi found Lassen 's theory untenable but had no theories of his own, only suggesting that either Newara is a vulgarism of sanskritic Nepala, or Nepala is Sanskritisation of the local ethnic.[28] Levi'
s view has some support in later works. [29][30][31] The idea that Nepal is a polished form of Newar, the name of the indigenous people of Kathmandu valley, may be gaining support, [24] but it leaves the question of etymology unanswered.One theory proposes that Nepa is a Tibeto - Burman stem consisting of Ne(cattle) and Pa(keeper), which alludes to the fact that early inhabitants of the valley were Gopalas(cowherds) and Mahispalas(buffalo - herds). [24] Suniti Kumar Chatterji thought that 'Nepal'
originated from Tibeto - Burman roots - Ne, of uncertain meaning(as multiple possibilities exist), and pala or bal, whose meaning is lost entirely. [32]

History
Main article: History of Nepal

Lumbini, listed as the birthplace of Gautama Buddha by the UNESCO World Heritage Convention
Ancient Nepal
Neolithic tools found in the Kathmandu Valley indicate that people have been living in the Himalayan region
for at least eleven thousand years. [33]

Nepal is mentioned in the late Vedic Atharvaveda Pariśiṣṭa and in the post - Vedic Atharvashirsha Upanishad. [34] Nepal is also mentioned in Hindu texts such as the Narayana Puja and the regional text "Nepal Mahatmya"
which claims to be a part of Skanda Purana. [34] The Gopal Bansa were likely one of the earliest inhabitants of Kathmandu valley. [35] The earliest rulers of Nepal were the Kiratas(Kirata Kingdom), peoples often mentioned in Hindu texts, who ruled Nepal
for many centuries.Various sources mention up to 32 Kirati kings ruling over 16 centuries. [36]

Around 500 BCE, small kingdoms and confederations of clans arose in the southern regions of Nepal.From one of these, the Shakya polity, arose a prince who later renounced his status to lead an ascetic life, founded Buddhism, and came to be known as Gautama Buddha(traditionally dated 563– 483 BCE). [37] By 250 BCE, the southern regions had come under the influence of the Maurya Empire of North India and later became a vassal state under the Gupta Empire in the 4 th century CE. [38] In Samudragupta 's Allahabad Pillar it is mentioned as a border country.

The kings of the Lichhavi dynasty have been found to have ruled Nepal after the Kirat monarchical dynasty.The context that "Suryavansi Kshetriyas had established a new regime by defeating the Kirats"
can be found in some genealogies and Puranas.It is not clear yet when the Lichhavi dynasty was established in Nepal.According to the opinion of Baburam Acharya, the prominent historian of Nepal, Lichhavies established their independent rule by abolishing the Kirati state that prevailed in Nepal around 250 CE.

The Licchavi dynasty went into decline in the late 8 th century, and was followed by a Newar or Thakuri era.Thakuri kings ruled over the country up to the middle of the 12 th century CE;
King Raghav Dev is said to have founded the ruling dynasty in October 869 CE. [39] King Raghav Dev also started the Nepal Sambat. [40]

Medieval Nepal
Main articles: Malla(Nepal) and Khasa kingdom
In the early 12 th century, leaders emerged in far western Nepal whose names ended with the Sanskrit suffix malla("wrestler").These kings consolidated their power and ruled over the next 200 years, until the kingdom splintered into two dozen petty states.Another Malla dynasty beginning with Jayasthiti emerged in the Kathmandu valley in the late 14 th century, and much of central Nepal again came under a unified rule.In 1482, the realm was divided into three kingdoms: Kathmandu, Patan, and Bhaktapur. [41]

Unification of Nepal
Main article: Kingdom of Nepal

King Tribhuvan giving an audience to British general Claude Auchinleck at the royal palace in Kathmandu, 1945

Prime Minister of Israel David Ben - Gurion and Prime Minister of Nepal B.P.Koirala
In the mid - 18 th century, Prithvi Narayan Shah, a Gorkha king, set out to put together what would become present - day Nepal.He embarked on his mission by securing the neutrality of the bordering mountain kingdoms.After several bloody battles and sieges, notably the Battle of Kirtipur, he managed to conquer the Kathmandu Valley in 1769. A detailed account of Prithvi Narayan Shah 's victory was written by Father Giuseppe, an eyewitness to the war.[42]

The Gorkha control reached its height when the North Indian territories of the Kumaon and Garhwal Kingdoms in the west to Sikkim in the east came under Nepalese control.A dispute with Tibet over the control of mountain passes and inner Tingri valleys of Tibet forced the Qing Emperor of China to start the Sino - Nepali War compelling the Nepali to retreat and pay heavy reparations to Peking.

Rivalry between the Kingdom of Nepal and the East India Company over the control of states bordering Nepal eventually led to the Anglo - Nepali War(1815– 16).At first, the British underestimated the Nepali and were soundly defeated until committing more military resources than they had anticipated needing.Thus began the reputation of Gurkhas as fierce and ruthless soldiers.The war ended in the Sugauli Treaty, under which Nepal ceded recently captured lands as well as the right to recruit soldiers.Madhesis, having supported the East India Company during the war, had their lands gifted to Nepal. [43]

Rana autocratic regime
Factionalism inside the royal family led to a period of instability.In 1846, a plot was discovered revealing that the reigning queen had planned to overthrow Jung Bahadur Kunwar, a fast - rising military leader.This led to the Kot massacre;
armed clashes between military personnel and administrators loyal to the queen led to the execution of several hundred princes and chieftains around the country.Jung Bahadur Kunwar emerged victorious and founded the Rana dynasty, later known as Jung Bahadur Rana.The king was made a titular figure, and the post of Prime Minister was made powerful and hereditary.The Ranas were staunchly pro - British and assisted them during the Indian Rebellion of 1857(and later in both World Wars).Some parts of the Terai region populated with non - Nepali peoples were gifted to Nepal by the British as a friendly gesture because of her military help to sustain British control in India during the rebellion.In 1923, the United Kingdom and Nepal formally signed an agreement of friendship that superseded the Sugauli Treaty of 1816.[43]

Legalized slavery was abolished in Nepal in 1924.[44] Rana rule was marked by tyranny, debauchery, economic exploitation and religious persecution. [45][46]

Democratic Nepal
In the late 1940 s, newly emerging pro - democracy movements and political parties in Nepal were critical of the Rana autocracy.Meanwhile, with the invasion of Tibet by China in the 1950 s, India sought to counterbalance the perceived military threat from its northern neighbour by taking pre - emptive steps to assert more influence in Nepal.India sponsored both King Tribhuvan(ruled 1911– 1955) as Nepal 's new ruler in 1951 and a new government, mostly comprising the Nepali Congress, thus terminating Rana hegemony in the kingdom.[43]

After years of power wrangling between the king and the government, King Mahendra(ruled 1955– 1972) scrapped the democratic experiment in 1959, and a "partyless"
Panchayat system was made to govern Nepal until 1989, when the "Jan Andolan"(People 's Movement) forced King Birendra (ruled 1972–2001) to accept constitutional reforms and to establish a multiparty parliament that took seat in May 1991.[47] In 1991–92, Bhutan expelled roughly 100,000 Bhutanese citizens of Nepali descent, most of whom have been living in seven refugee camps in eastern Nepal ever since.

    In 1996, the Communist Party of Nepal started a violent bid to replace the royal parliamentary system with a people 's republic. This led to the long Nepali Civil War and more than 12,000 deaths. On 1 June 2001, there was a massacre in the royal palace. King Birendra, Queen Aishwarya and seven other members of the royal family were killed. The alleged perpetrator Crown Prince Dipendra, who allegedly committed suicide shortly thereafter, was briefly declared king for three days while he was in coma. Following the carnage, King Birendra'
    s brother Gyanendra inherited the throne.On 1 February 2005, King Gyanendra dismissed the elected government and legislature, assuming full executive powers to quash the violent Maoist movement. [47] But this initiative was unsuccessful because a stalemate had developed in which the Maoists were firmly entrenched in large expanses of countryside but could not yet dislodge the military from numerous towns and the largest cities.In September 2005, the Maoists declared a three - month unilateral ceasefire to negotiate.

    In response to the 2006 democracy movement, King Gyanendra agreed to relinquish sovereign power to the people.On 24 April 2006 the dissolved House of Representatives was reinstated.Using its newly acquired sovereign authority, on 18 May 2006 the House of Representatives unanimously voted to curtail the power of the king and declared Nepal a secular state, ending its time - honoured official status as a Hindu Kingdom.On 28 December 2007, a bill was passed in parliament to amend Article 159 of the constitution– replacing "Provisions regarding the King"
    by "Provisions of the Head of the State"–
    declaring Nepal a federal republic, and thereby abolishing the monarchy. [48] The bill came into force on 28 May 2008.[49]


    Dr.Ram Baran Yadav, the first President of Nepal Following the declaration of the federal republic, an election was held
    for the Constituent Assembly that would draft a new constitution. [50] A period of instability followed; with changing governments, and various nationalist movements and popular protests demanding
    for ethnic autonomy; the political deadlock meant the constituent assembly failed to adopt a constitution within the stipulated time.The Constituent Assembly was dissolved in May 2012.[51] A second election
    for a new Constituent Assembly was held in 2013 under a non - partisan government led by former Chief Justice Khil Raj Regmi. [52][53] On 25 April 2015, a magnitude 7.8 earthquake struck Nepal[54] followed by a 7.3 magnitude aftershock two weeks later, causing a combined death toll of 8, 500, about 21, 000 injuries and material loss amounting to a third of the country 's annual Gross Domestic Product.[55] The Constitution of Nepal, passed with a 90% majority was announced in 20 September 2015 making Nepal a federal democratic republic divided into seven unnamed provinces. It was, however, rejected by the Madhesi nationalist parties, who intensified their protests, leading to an unofficial economic blockade by the Government of India. By February 2016, an amendment had been agreed between India and Nepal, and the Madhesis slowly backed down after it was passed by parliament.[56][57] The elections for the local, provincial and federal levels of government were held in 2017 and Nepal Communist Party emerged as the ruling party with a strong majority at the federal level, as well as six of the seven provinces.[58]

    Geography Main articles: Geography of Nepal and Geology of Nepal

    A topographic map of Nepal.

    Mount Everest, the highest peak on earth, lies on the Nepal - China border.Nepal is of roughly trapezoidal shape, about 800 kilometres(500 mi) long and 200 kilometres(120 mi) wide, with an area of 147, 181 km2(56, 827 sq mi).It lies between latitudes 26° and 31° N, and longitudes 80° and 89° E.Nepal 's defining geological processes began 75 million years ago when the Indian plate, then part of the southern supercontinent Gondwana, began a north-eastward drift caused by seafloor spreading to its south-west, and later, south and south-east.[59] Simultaneously, the vast Tethyn oceanic crust, to its northeast, began to subduct under the Eurasian plate.[59] These dual processes, driven by convection in the Earth'
    s mantle, both created the Indian Ocean and caused the Indian continental crust eventually to under - thrust Eurasia and to uplift the Himalayas. [59] Immediately south of the emerging Himalayas, plate movement created a vast trough that rapidly filled with river - borne sediment[60] and now constitutes the Indo - Gangetic Plain. [61] Nepal lies almost completely within this collision zone, occupying the central sector of the Himalayan arc, nearly one third of the 2, 400 km(1, 500 mi) - long Himalayas, [62][63][64][65][66][67] with a small strip of southernmost Nepal stretching into the Indo - Gangetic plain and two districts in the northwest stretching up to the Tibetan plateau.

    Nepal is divided into three principal physiographic belts known as Himal - Pahad - Terai. [d] Himal is the mountain region containing snow and situated in the Great Himalayan Range; it makes up the northern part of Nepal.It contains the highest elevations in the world including 8, 848 metres(29, 029 ft) height Mount Everest(Sagarmāthā in Nepali) on the border with China.Seven other of the world 's "eight-thousanders" are in Nepal or on its border with China: Lhotse, Makalu, Cho Oyu, Kangchenjunga, Dhaulagiri, Annapurna and Manaslu. Pahad is the mountain region that does not generally contain snow. The mountains vary from 800 to 4,000 metres (2,600 to 13,100 ft) in altitude, with progression from subtropical climates below 1,200 metres (3,900 ft) to alpine climates above 3,600 metres (11,800 ft). The Lower Himalayan Range, reaching 1,500 to 3,000 metres (4,900 to 9,800 ft), is the southern limit of this region, with subtropical river valleys and "hills" alternating to the north of this range. Population density is high in valleys but notably less above 2,000 metres (6,600 ft) and very low above 2,500 metres (8,200 ft), where snow occasionally falls in winter. The southern lowland plains or Terai bordering India are part of the northern rim of the Indo-Gangetic Plain. Terai is the lowland region containing some hill ranges. The plains were formed and are fed by three major Himalayan rivers: the Koshi, the Narayani, and the Karnali as well as smaller rivers rising below the permanent snowline. This region has subtropical to tropical climate. The outermost range of the foothills called Sivalik Hills or Churia Range, cresting at 700 to 1,000 metres (2,300 to 3,280 ft), marks the limits of the Gangetic Plain; however broad, low valleys called Inner Terai Valleys (Bhitri Tarai Upatyaka) lie north of these foothills in several places.

    The Indian plate continues to move north relative to Asia at about 50 mm(2.0 in ) per year. [68] This makes Nepal an earthquake prone zone, and periodic earthquakes that have devastating consequences present a significant hurdle to development.Erosion of the Himalayas is a very important source of sediment, which flows to the Indian Ocean. [69] Saptakoshi, in particular, carries huge amount of silt out of Nepal but sees extreme drop in Gradient in Bihar, causing severe floods and course changes, and is therefore, known as the sorrow of Bihar.Severe flooding and landslides cause deaths and disease, destroy farmlands and cripple the transport infrastructure of the country, during the monsoon season each year.

    Nepal has five climatic zones, broadly corresponding to the altitudes.The tropical and subtropical zones lie below 1, 200 metres(3, 900 ft), the temperate zone 1, 200 to 2, 400 metres(3, 900 to 7, 900 ft), the cold zone 2, 400 to 3, 600 metres(7, 900 to 11, 800 ft), the subarctic zone 3, 600 to 4, 400 metres(11, 800 to 14, 400 ft), and the Arctic zone above 4, 400 metres(14, 400 ft).Nepal experiences five seasons: summer, monsoon, autumn, winter and spring.The Himalayas block cold winds from Central Asia in the winter and form the northern limits of the monsoon wind patterns.

    Biodiversity Main article: Wildlife of Nepal See also: Protected areas of Nepal and Community forestry in Nepal Land cover map of 2010 This land cover map of Nepal using Landsat 30 m(2010) data shows forest cover is the dominant type of land cover in Nepal[70]

    The greater one - horned rhinoceros roams the sub - tropical grasslands of the Terai plains.Nepal contains a disproportionately large diversity of plants and animals, relative to its size. [71][72] Nepal, in its entirety, forms the western portion of the eastern Himalayan biodiversity hotspot, with notable biocultural diversity. [73] The dramatic differences in elevation found in Nepal(60 m from sea level in the Terai plains, to 8, 848 m Mount Everest)[74] result in a variety of biomes. [71] Eastern half of Nepal is richer in biodiversity as it receives more rain, compared to western parts, where arctic desert - type conditions are more common at higher elevations. [72] Nepal is a habitat
    for 4.0 % of all mammal species, 8.9 % of bird species, 1.0 % of reptile species, 2.5 % of amphibian species, 1.9 % of fish species, 3.7 % of butterfly species, 0.5 % of moth species and 0.4 % of spider species. [72] In its 35 forest - types and 118 ecosystems, [71][e] Nepal harbours 2 % of the flowering plant species, 3 % of pteridophytes and 6 % of bryophytes. [72]

    Nepal 's forest cover is 59,624 km2 (23,021 sq mi), 40.36% of the country'
    s total land area, with an additional 4.38 % of scrubland,
    for a total forested area of 44.74 % , an increase of 5 % since the turn of the millennium. [75] In the southern plains, Terai - Duar savanna and grasslands ecoregion contains some of the world 's tallest grasses as well as Sal forests, tropical evergreen forests and tropical riverine deciduous forests.[76] In the lower hills (700 m – 2,000 m), subtropical and temperate deciduous mixed forests containing mostly Sal (in the lower altitudes), Chilaune and Katus, as well as subtropical pine forest dominated by Chir pine are common. The middle hills (2,000 m – 3,000 m) are dominated by Oak and Rhododendron. Subalpine coniferous forests cover the 3,000 m to 3,500 m range, dominated by Oak (particularly in the west), Eastern Himalayan fir, Himalayan pine and Himalayan hemlock; Rhododendron is common as well. Above 3,500 m in the west and 4,000 m in the east, coniferous trees give way to Rhododendron-dominated alpine shrubs and meadows.[72]

    Among the notable trees, are the astringent Azadirachta indica, or neem, which is widely used in traditional herbal medicine, [77] and the luxuriant Ficus religiosa, or peepal, [78] which is displayed on the ancient seals of Mohenjo - daro, [79] and under which Gautam Buddha is recorded in the Pali canon to have sought enlightenment. [80] Rhododendron is the national flower of Nepal. [81]

    Most of the subtropical evergreen broad - leaved forest of the lower himalayan region is descended from the tethyan tertiary flora. [82] As the Indian plate collided with Eurasia forming and raising the Himalayas, the arid and semi - arid mediterranean flora was pushed up and adapted to the more alpine climate over the next 40– 50 million years. [82][83] The Himalayan biodiversity hotspot was the site of mass exchange and intermingling of the Indian and Eurasian species in the neogene. [84] One mammal species(Himalayan field mouse), two each of bird and reptile species, nine amphibian, eight fish and 29 butterfly species are endemic to Nepal. [72][f]


    Himalayan monal(Danphe), the national bird of Nepal, [86] nests high in the himalayas Nepal contains 107 IUCN - designated threatened species, 88 of them animal species, 18 plant species and one species of "fungi or protist"
    group. [87] These include the endangered Bengal tiger, the Red panda, the Asiatic elephant, the Himalayan musk deer, the Wild water buffalo and the South Asian river dolphin, [88] as well as the critically endangered Gharial, the Bengal florican, [71][89] and the White - rumped Vulture, which has become nearly extinct by having ingested the carrion of diclofenac - treated cattle. [90] The pervasive and ecologically devastating human encroachment of recent decades has critically endangered Nepali wildlife.In response, the system of national parks and protected areas, first established in 1973 with the enactment of National Parks and Wildlife Conservation Act 1973, [91] was substantially expanded.Vulture restaurants[72] coupled with a ban on veterinary usage of diclofenac has seen a rise in the number of white - rumped vultures. [92][90] The community forestry program which has seen a third of the country 's population directly participate in managing a quarter of the total forested area, has helped the local economies while reducing human-wildlife conflict.[93][94] The breeding programmes[95] coupled with community-assisted military patrols,[96] and a crackdown on poaching and smuggling, has seen poaching of critically endangered tigers and elephants as well as vulnerable rhinos, among others, go down to effectively zero, and their numbers have steadily increased.[97] Nepal has ten national parks, three wildlife reserves, one hunting reserve, three conservation areas and eleven buffer zones, covering a total area of 28,959.67 km2 (11,181.39 sq mi), or 19.67% of the total land area,[98] while ten wetlands are registered under the Ramsar Convention.[99]

    Politics and government Politics Main article: Politics of Nepal Main office holders Bidhya Devi Bhandari Bidhya Devi Bhandari, President of Nepal since 29 October 2015 Khadga Prasad Oli Khadga Prasad Oli, Prime Minister since 15 February 2018 Nepal is a parliamentary republic with a multi - party system. [100] It has four political parties recognised in the federal parliament: Nepal Communist Party(NCP), Nepali Congress(NC), [100] Samajbadi Party Nepal(SPN) and Rastriya Janata Party Nepal(RJPN). [100] While all major parties officially espouse democratic socialism, NCP is considered leftist
    while Nepali Congress is considered centrist, with most considering it center - left and some center - right. [101] The minor party SPN is leftist and RJPN is center - right to right - wing. [102] During most of the brief periods of democratic exercise in the 1950 s as well as the 1990 s, Nepali Congress held a majority in parliament. [103] Following the entry of the Maoists into the political process, they were the largest party in the first constituent assembly and Nepali Congress was the largest in the second, with no party winning a majority. [104] In the aftermath of the 2017 elections, the first one according to the new constitution, NCP has become the ruling party at the federal level as well as six out of seven provinces. [105] While Nepali Congress has a significantly reduced representation, it is the only major opposition to the ruling communist party in all levels of government. [106]

    Early politics in the Kingdom of Nepal was characterised by factionalism, conspiracies and murders, including two major massacres. [g] After almost a century of power - wrangling among the prominent Basnyat, Pande and Thapa families, a fast - rising military leader Bir Narsingh Kunwar[h] emerged on top in the aftermath of the Kot massacre, and established the Rana autocratic regime which consolidated powers of the King as well as prime minister and reigned
    for another century, with a policy of oppression and isolationism. [107][108] By the 1930 s, Nepali expatriates in India had started smuggling in writings on political philosophies, which gave birth to a vibrant underground political movement in the capital, birthing Nepal Praja Parishad in 1939, which was dissolved only two years later, following the execution of the four great martyrs.Around the same time, Nepalis involved in the Indian Independence Movement started organising into political parties, leading to the birth of Nepali Congress and Communist Party of Nepal.Following Indian Independence, Nepali Congress was successful in overthrowing the Rana regime with support from the Indian government and cooperation from the king. [109] While communism was still trying to find its footing, Nepali Congress enjoyed overwhelming support of the electorate.Following a brief ten - year exercise in democracy, another partyless autocracy was initiated, this time by the King, who deposed the democratically elected government of Nepali Congress, imposed or exiled prominent leaders and issued a ban on party politics. [109][110][111][103]

    Many political parties and their leaders remained underground or in exile
    for the next 30 years of partyless politics in Nepal. [112] BP Koirala was released from prison in 1968 and went into exile in Benaras, returning in 1976 only to immediately be put in house arrest. [109] Although an armed insurgency launched by the major communist faction called the Jhapa movement had failed comprehensively by 1971, it formed the foundation
    for the dominant communist power, CPN ML, that was officially launched in 1978.[113] A general referendum was held in 1980, which saw the CPN ML campaign
    for the option of multi - party democracy, along with Nepali Congress, but the Panchayat System was declared the winner to significant controversy. [111] The Panchayat rule saw governments led by a group of monarchy loyalists taking turns, with Surya Bahadur Thapa, Tulsi Giri and Kirti Nidhi Bista becoming prime minister three times each, among others.It introduced a number of reforms, built infrastructures and modernised the country,
    while significantly curtailing political freedom, imposing the Nepali language and khas culture to the oppression of all others, and spreading Indophobic propaganda the effects of which are experienced to the present day. [109][103]


    Prachanda speaking at a rally in Pokhara.In 1990, the joint civil resistance launched by the United left front and Nepali Congress was successful in overthrowing the Panchayat, and the country became a constitutional monarchy. [109][114] The United Left Front became CPN UML. [103] The Panchayat loyalists formed National Democratic Party which emerged as the third major party.While Nepali Congress ran the government
    for most of the next ten years of democracy that followed, democracy was mostly a disappointment owing to the immature democratic culture and political infighting in the capital, as well as the civil war that followed the guerrilla insurgency launched by the Maoist Party.Following a four - year autocratic rule by King Gyanendra that failed to defeat the Maoists, a mass civil protest was launched by a coalition of the maoists and the political parties in 2006, which forced the king to stepped down, brought the maoists to the peace process, and established a democratic republic by 2008.[115][116]

    Following the political consensus to draft the new constitution of the Republic via a constituent assembly, Nepali politics saw a rise of nationalist groups and ideologies.While the political power - wrangling caused continuous instability, maintaining the established average of nine months per government, this period saw two constituent assembly elections and the rise of Madhesi nationalist parties, especially in the Eastern Terai region. [104][117][118][119] By 2015, the new constitution had been promulgated and Nepal became "a federal democratic republic striving towards democratic socialism". [120] In 2017, a series of elections were held according to the new constitution, which established Nepal Communist Party(NCP)(formally united after the election) as the ruling party at the federal level as well as six of the seven provinces, Nepali Congress as the only significant opposition in federal and provincial levels,
    while the Madhesi coalition formed the provincial government in Province No .2, but boasts negligible presence in the rest of the country. [121][122][123]

    Government Main articles: Government of Nepal and Constitution of Nepal

    Entrance to Singha Durbar, the seat of the Nepali government in Kathmandu Nepal is governed according to the Constitution of Nepal, which came into effect on 20 September 2015. It defines Nepal as having multi - ethnic, multi - lingual, multi - religious, multi - cultural characteristics with common aspirations of people living in diverse geographical regions, and being committed to and united by a bond of allegiance to the national independence, territorial integrity, national interest, and prosperity of Nepal.All Nepali people collectively constitute the state.

    The Government of Nepal comprises three branches: [20]

    Executive: The form of governance of Nepal is a multi - party, competitive, federal democratic republican parliamentary system based on plurality.The executive power of Nepal rests with the Council of Ministers in accordance with the Constitution and Nepali law.The President appoints the parliamentary party leader of the political party with the majority in the House of Representatives as a Prime Minister, and a Council of Ministers is formed in his / her chairmanship.The executive power of the provinces, pursuant to the Constitution and laws, is vested in the Council of Ministers of the province.The executive power of the province shall be exercised by the province Head in
    case ofabsence of the province Executive in a State of Emergency or enforcement of Federal rule.Every province has a ceremonial Head as the representative of the Federal government.The President appoints a Governor
    for every province.The Governor exercises the rights and duties as specified in the constitution or laws.The Governor appoints the leader of the parliamentary party with the majority in the Provincial Assembly as the Chief Minister and the Council of Ministers are formed under the chairpersonship of the Chief Minister.Legislature: The Legislature of Nepal, called Federal Parliament, consists of two Houses, namely the House of Representatives and the National Assembly.The term of House of Representatives is five years.The House of Representatives consists of 275 members: 165 members elected through the first - past - the - post electoral system consisting of one member from each of the one hundred and sixty five electoral constituencies; 110 elected from proportional representation electoral system where voters vote
    for parties,
    while treating the whole country as a single electoral constituency.The National Assembly is a permanent house.The tenure of members of National Assembly is six years.The National Assembly consists of 59 members: 56 members elected from an Electoral College, comprising members of provincial Assembly and chairpersons and vice - chairpersons of Village councils and Mayors and Deputy Mayors of Municipal councils, with different weights of votes
    for each, with eight members from each province, including at least three women, one Dalit, and one person with a disability or a member of a minority.Three members, including at least one woman, are to be nominated by the President on the recommendation of the Government of Nepal.A Provincial Assembly is the unicameral legislative assembly
    for a federated province. [124] The term
    for the Provincial Assembly is five years.Judiciary: Powers relating to justice in Nepal are exercised by courts and other judicial institutions in accordance with the provisions of the constitution, other laws, and recognised principles of justice.Nepal has a unitary three - tier independent judiciary that comprises the Supreme Court, headed by the Chief Justice of Nepal, 7 High Courts, and a large number of trial courts.The supreme court is the highest court in the land.The high court is the highest court in each province.There are district courts, one in each district below the high courts.The local governments may convene local judicial bodies to resolve disputes and render non - binding verdicts on cases not involving actionable crime.The actions and proceedings of the local judicial bodies may be guided and countermanded by the district courts. [20] Administrative divisions Main article: Administrative divisions of Nepal SudurpashchimKarnaliProvince 5 GandakiBagmatiProvince 2 Province 1 CHINAINDIA Nepal is a federal republic comprising 7 provinces.Each province is composed of 8 to 14 districts.The districts, in turn, comprise local units known as urban and rural municipalities. [20] There is a total of 753 local units which includes 6 metropolitan municipalities, 11 sub - metropolitan municipalities and 276 municipalities
    for a total of 293 urban municipalities, and 460 rural municipalities. [125] Each local unit is composed of wards.There are 6, 743 wards in total.

    The local governments enjoy executive and legislative as well as limited judicial powers in their local jurisdiction.The provinces have unicameral parliamentary westminster system of governance.The local and provincial governments exercise some absolute powers and some powers shared with provincial and federal governments as applicable, as listed in the constitution of Nepal.The laws enacted by local governments may not contradict existing laws at the provincial and federal levels or the national constitution.Similarly, provincial legislature may not enact laws contradicting federal laws or the national constitution.The powers not listed in the constitution are exercised by the federal government.The district coordination committee, a committee composed of all elected officials from the local governments in the district, has a very limited role. [20][125]

    No.Provinces Capital Districts Area(km2) Population(2011) Density(people / km2) 1 Province No .1 Biratnagar 14 25, 905 km2 4, 534, 943 175 2 Province No .2 Janakpur 8 9, 661 km2 5, 404, 145 559 3 Bagmati Pradesh Hetauda 13 20, 300 km2 5, 529, 452 272 4 Gandaki Pradesh Pokhara 11 21, 504 km2 2, 413, 907 112 5 Province No .5 Butwal 12 22, 288 km2 4, 891, 025 219 6 Karnali Pradesh Birendranagar 10 27, 984 km2 1, 168, 515 41 7 Sudurpashchim Pradesh Godavari 9 19, 539 km2 2, 552, 517 130 Total Nepal Kathmandu 77 147, 181 km2 26, 494, 504 180 Laws and law enforcement Main article: Law enforcement in Nepal The Constitution of Nepal is the supreme law of the land, and any other laws contradicting it are automatically invalid to the extent of the contradiction. [126] The specific legal provisions are codified as Civil Code and Criminal Code, accompanied by Civil Procedure Code and Criminal Procedure Code respectively. [127] Other laws may be enacted by the parliament at all levels of government to supplement but not supersede these laws and other laws enacted by the higher level parliaments, as applicable.The Supreme Court is the highest authority in the interpretation of laws and it can direct the parliament to amend or enact new laws as required.Nepali laws are considered generally more progressive compared to other third world countries, and in some instances, even some countries of the first world.Nepal has abolished the death penalty. [128] It also has made progress in LGBT rights and gender equality.It recognises marital rape and supports abortion rights; however because of the rise in sex - selective abortion, constraints have been introduced.Nepal is a signatory to the Geneva Convention, Conventions / Treaties on the prohibition of Biological, Chemical and Nuclear weapons, [129] International Labour Organisation Fundamental Conventions, Treaty on the non - proliferation of nuclear weapons as well as the Paris climate accord.Some legal provisions that are guided by socio - economic, cultural and religious sensibilities remain discriminatory.There is gender based discrimination against foreign nationals married to Nepali citizens. [i] Paternal lineage of a person is valued and required in legal documents.Many laws remain unenforced in practice.

    Nepal Police is the primary law enforcement agency of Nepal.It is an independent organisation under the command of the Inspector General of Police, who is appointed by and reports to the Home ministry.Nepal police is responsible
    for maintaining law and order in the country.It is also responsible
    for traffic management in the country, which is done by the special subdivision, the Nepal traffic police force.Nepal Armed Police Force, a separate paramilitary police organisation works in cooperation with Nepal police, assisting the latter in maintaining checkpoints, border security, patrols, crowd control in
    case ofviolent protests, security of vital assets, counter - insurgency and anti - terrorism actions, as well as any other internal security matter that requires use of force.The Crime Investigation Department is the special branch of Nepal Police that specialises in criminal investigation and forensic analysis, which maintains an investigative team in every district of the country and is called in when normal police procedures and investigations prove insufficient in resolving a
    case.The National Investigation Department of Nepal is an independent intelligence organisation which gathers intelligence relevant to internal security and law enforcement, as part of its mandate. [131][132][133][134][135] The Commission
    for the Investigation of Abuse of Authority is an independent investigative agency that investigates and prosecutes cases related to corruption and bribery, in addition to the abuse of authority by government officials and officeholders.Nepal is ranked 76 out of 163 countries in Global Peace Index(GPI) as of 2019.[136]

    Foreign economic and strategic relations Main articles: Foreign relations of Nepal and Nepalese Armed Forces

    Gurkha Memorial, London Nepal pursues a policy of "balanced relations"
    with the two giant immediate neighbours, India and China, [137][138] even though it shares an unparalleled socio - cultural ties with India[139][140] and the Treaty of Peace and Friendship between two provides
    for a much closer relationship between them. [141] Nepal and India share an open border with free movement of people, religious, cultural and marital ties, and each boasts Hindu and Buddhist pilgrimages widely visited by citizens of the other.Nepal 's currency is permanently pegged to the Indian currency and most of the third-country trade of Nepal is carried out via Indian ports. Millions of people from Nepal reside in India for education and work, and vice versa. India is Nepal'
    s largest trading partner, and Nepal imports all of its oil and gas, and almost all of a number of other essential supplies including medicine, from India.Nepalis, prominently, participated in the Indian Independence Movement, and India closely monitors and seeks to influence Nepal 's internal politics, which has led to India playing indispensable roles in all of Nepal'
    s democratic movements,
    while on some occasions, India 's unwelcome interventions in Nepal'
    s politics has given rise to a significant skepticism regarding India 's intentions and strengthened anti-India sentiment in Nepal. Nepalis serve in the Gurkha battalions of the Indian army and have fought in India'
    s wars.Nepalis can own property in India,
    while Indians are free to live and work in Nepal. [139] While Nepal votes independently in international forums, and India and Nepal often disagree in such votes, Nepal has made a commitment to seek India 's approval on any purchase of military weapons from third countries,[142] while India unilaterally considers smaller countries in South Asia including Nepal as under its security sphere, requiring military protection.[143][144] Nepal'
    s long - term grievances against India stem from India 's interference in Nepal'
    s internal politics, the 1950 treaty that is seen as unfair to Nepal, [141] alleged border encroachment and harassment by Indian border security force, flooding caused by river dams controlled by India, and India 's willingness to blockade Nepal to force its agenda. India has expressed concern with Nepal'
    s apathy toward alleged cross - border terrorism carried out by Pakistani terrorists based in Nepal, Nepal 's refusal to acknowledge support for India during India-China conflict and Nepal'
    s recognition of Chinese claim over Tibet, and the alleged use of anti - India propaganda by Nepali politicians
    for political gain, among others.


    Nepal is one of the major contributors to UN peacekeeping missions.Nepal established diplomatic relations with the People 's Republic of China on 1 August 1955, signing the Treaty of Peace and Friendship in 1960, and relations since have been based on the Five Principles of Peaceful Coexistence. Nepal has always maintained neutrality in conflicts between China and India. It remains firmly committed to the One China Policy, and is known to curb anti-China activities from the Tibetan refugees in Nepal.[145][146] Citizens of both countries can cross the border and travel as far as 30 km without a visa.[147] China is viewed favorably in Nepal because of the latter'
    s assistance in infrastructure development, absence of any border disputes or serious interference in internal politics, aid during natural disasters, and favorability has increased since 2015, when China helped Nepal during the economic blockade imposed by India. [148] Since then, Nepal and China have moved closer on trade and connectivity, with China granting Nepal access to its ports
    for third country trade, and Nepal joining China 's ambitious Belt and Road Initiative with plans for expansive road and railway projects.[149]

    Nepal emphasizes greater cooperation in the South Asia region and actively pushed
    for the establishment of SAARC, the South Asian Association
    for Regional Cooperation, the permanent secretariat of which, is hosted in Kathmandu. [150] Nepal was one of the first countries to recognise an independent Bangladesh, and the two countries seek to enhance greater cooperation, on trade and water management; seaports in Bangladesh which are closer to Nepal, are seen as viable alternatives to India 's monopoly on Nepal'
    s third country trade. [151] Nepal 's relationship with Bhutan has become acrimonious since Bhutan carried out an ethnic cleansing against its citizens of Nepali origin in the early 1990s,[152][153] most of whom have been resettled in third countries after decades of efforts to repatriate them failed.[154] Nepal was the first South Asian country to establish diplomatic relations with Israel, and the countries enjoy a strong relationship.[155] However, Nepal also recognises the rights of the Palestinians, and has voted in favor of recognising Palestine at the UN and against the recognition of Jerusalem as Israel'
    s capital. [156] Other countries that Nepal maintains a close relationship with, includes countries that are the most generous donors and development partners of Nepal, namely, the United States, the United Kingdom, Denmark, Japan and Norway, among others. [157]


    The multipurpose Kukri knife(top) is the signature weapon of Nepali armed forces, and is used by Indian and British Gurkhas, Nepal army, police and even Nepalese security guards.The President is the supreme commander of the Nepalese Army, although its routine management is handled by the Ministry of Defence.Nepal 's military expenditure for 2018 was $398.5 million,[158] around 1.4% of its GDP.[159] The age of qualification for the military service which is voluntary, is 18 years.[160] Nepal Army, an almost exclusively ground infantry force, which numbers at less than one hundred thousand,[161][162][163] has few aircraft, mainly helicopters, primarily used for transport, patrol and search and rescue.[164] Directorate of Military Intelligence is the military intelligence agency under Nepal Army,[165] while National Investigation Department is the independent intelligence agency tasked with national and international intelligence gathering.[161] As the military of an underdeveloped small country sandwiched between two superpowers, Nepal Army is not maintained as a national defence force against foreign aggression, but rather for routine security of critical assets, anti-poaching activities in national parks, internal security in times of crisis, like civil war, terrorism, etc. and for search and rescue during natural disasters.[166] Nepal Army also undertakes construction projects.[167] While there are no officially sanctioned discriminatory policies on recruitment, except on the basis of physical or mental fitness and age, but reservations for marginalised groups instead,[168] the army is dominated by the elite Pahari warrior castes, women form an abysmal minority[169] and there are allegations of discrimination against sexual minorities,[170] in practice. Nepal mainly depends on diplomacy for national defence. Nepal has always maintained a policy of neutrality between its neighbours, has amicable relations with other countries in the region, and has pursued a policy of non-alignment at the global stage. In addition to the SAARC and the UN, Nepal is also a member of WTO, BIMSTEC and ACD, among others. It conducts joint military exercises with China, India and the United States. Nepal has bilateral diplomatic relations with 167 countries and the EU,[171] has embassies in 30 countries[172] and six consulates,[173] while 25 countries maintain their embassies in Nepal, and more than 80 other countries maintain non-residential diplomatic missions.[174] Nepal is one of the major contributors to the UN Peacekeeping Missions having contributed more than 119,000 personnel to 42 missions since 1958.[175] Nepali people have a reputation for honesty, loyalty and bravery, which has led to Nepalese citizens serving as the legendary Gurkha warriors in the Indian and British armies for the last 200 years, who have fought in both world wars, India-Pakistan wars as well as being deployed in both Afghanistan and Iraq,[176] even though Nepal was not directly involved in any of those conflicts, and winning the highest military awards including the Victoria Cross and the Param Vir Chakra.[177][178] Many Nepali migrant workers are recruited as security guards, while Singapore Police Force also maintains a Gurkha contingent of Nepali recruits, owing to the same legendary reputation of the Nepali people.[179][180][181]

    Economy Main article: Economy of Nepal

    Terraced rice farming in Nepal

    Nepalese Chamber of Commerce, Lhasa, 1955

    Kathmandu street vendors

    Nepalese silver currency, 1695 Nepal 's gross domestic product (GDP) for 2018 was estimated at $28.8 billion.[182] With an annual growth rate calculated at 6.3% in 2018,[183] and expected to reach 7.1% in 2019,[184] Nepal is one of the fastest growing economies in the world. However, the country ranks 165th in the world[j] in nominal GDP per capita[185] and 162nd[k] in GDP per capita at PPP.[186] Nepal has been a member of WTO since 23 April 2004.[187]

    The 16.8 - million - worker Nepali labour force is the 37 th largest in the world, as of 2017.[188] The primary sector makes up 27.59 % of GDP, the secondary sector 14.6 % , and the tertiary sector 57.81 % . [189] Nepal 's foreign exchange remittances of US$8.1 billion in 2018, the 19th largest in the world and constituting 28.0% of GDP,[190] were contributed to its economy by millions of workers primarily in India, the middle east and East Asia, almost all of them unskilled labourers.[191][192] Major agricultural products include cereals (barley, maize, millet, paddy and wheat), oilseed, potato, pulses, sugarcane, jute, tobacco, milk and water buffalo meat.[193][194] Major industries include tourism, carpets, textiles, cigarettes, cement, brick, as well as small rice, jute, sugar and oilseed mills.[193] Nepal'
    s international trade greatly expanded in 1951 with the establishment of democracy; liberalisation began in 1985 and picked up pace after 1990. By the fiscal year 2016 / 17, Nepal 's foreign trade amounted Rs 1.06 trillion, a twenty-three folds increase from Rs 45.6 billion in 1990/91. More than 60% of Nepal'
    s trade is with India.Major exports include readymade garment, carpet, pulses, handicrafts, leather, medicinal herbs, and paper products, which account
    for 90 % of the total.Major imports include various finished and semi - finished goods, raw materials, machinery and equipment, chemical fertilizers, electrical and electronic devices, petroleum products, gold, and readymade garments. [195] Inflation was at 4.5 % in 2019.[184] Foreign exchange reserves were at US$9 .5 billion in July 2019, equivalent to 7.8 months of imports. [184]

    Nepal has made significant progress in poverty reduction bringing the population below the international poverty line(US$1 .90 per person per day) from 15 % in 2010 to just 9.3 % in 2018, although vulnerability remains extremely high, with almost 32 % of the population living on between US$1 .90 and US$3 .20 per person per day. [184] Nepal has made improvement in sectors like nutrition, child mortality, electricity, improved flooring and assets.If the progress of reducing poverty continues at this rate, then it is predicted that Nepal will halve the current poverty rate and eradicate it within the next 20 years. [196][197] The income distribution remains grossly uneven. [198] The agriculture sector is particularly vulnerable as it is highly dependent on the monsoon rains, with just 28 % of the arable land being irrigated, As of 2014.[199] Agriculture employs 76 % of the workforce, services 18 % , and manufacturing and craft - based industry 6 % . [200] Private investment, consumption, tourism and agriculture are the principal contributors to economic growth. [184]

    The government 's budget is about $13.71 billion (FY 2019/20),[201] but expenditure of infrastructure development budget usually fails to meet the target, and most of the development budget is hurriedly spent in the last months of the fiscal year when the monsoon and hurried work results in sub-par quality of construction.[202] The Nepali rupee has been tied to the Indian rupee at an exchange rate of 1.6 for many years. Since the loosening of exchange rate controls in the early 1990s, the black market for foreign exchange has all but disappeared. A long-standing economic agreement underpins a close relationship with India. The country receives foreign aid from the UK,[203][204] India, Japan, the US, the EU, China, Switzerland, and Scandinavian countries. Per capita income is $1,004.[205] The distribution of wealth among the Nepalis is consistent with that in many developed and developing countries: the highest 10% of households control 39.1% of the national wealth and the lowest 10% control only 2.6%. European Union (EU) (46.13%), the US (17.4%), and Germany (7.1%) are its main export partners. The European Union has emerged the largest buyer of Nepali ready-made garments (RMG). Exports to the EU accounted for "46.13 percent of the country'
    s total garment exports ".[206] Nepal's import partners include India (47.5%), the United Arab Emirates (11.2%), China (10.7%), Saudi Arabia (4.9%), and Singapore (4%).

    Besides having landlocked, rugged geography, few tangible natural resources and poor infrastructure, the ineffective post - 1950 government and the long - running civil war are also factors in stunting the country 's economic growth and development.[207][208][209] Debt bondage even involving debtors'
    children has been a persistent social problem in the western hills and the Terai, with an estimated 234, 600 people or 0.82 % of the population considered as enslaved, by The Global Slavery Index in 2016.[210]

    Tourism Main article: Tourism in Nepal Tourism is one of the largest industries in Nepal, employing more than a million people and contributing 7.9 % of the total GDP. [211] It is also one of the fastest - growing.The number of international visitors crossed one million in 2018
    for the first time(not counting Indian tourists arriving by land), [211] a 59 % increase from 736, 000 in the Nepal Tourism Year 2011, despite setbacks from the devastating 2015 earthquake. [212] Domestic tourism has witnessed a sharp increase since the earthquake, contributing 56 % of the total tourism earnings in 2018.[211] Nepal is home to four world heritage sites: Lumbini, the birthplace of Gautam Buddha, Sagarmatha National Park which includes Mount Everest, the highest peak on earth, seven monuments in the Kathmandu Valley collectively listed as one, and Chitwan National Park.Most of the country 's tourism is confined to these destinations, in addition to Pokhara, the Annapurna trekking circuit, and other Himalayan mountains which attract mountaineers and sightseers from around the world.[211] Although Nepal is home to eight of the fourteen eight-thousanders, all among the ten tallest mountains in the world, most of Nepal'
    s mountaineering earnings comes from Mt Everest, which is more accessible from the Nepalese side.Despite a vast potential
    for spiritual and cultural as well as mountaineering and eco - tourism, Nepal 's share of foreign tourists visiting South Asia is only about 6%, and tourists spend much less on average, with Nepal sharing only 1.7% of the total tourism earnings of South Asia.[213] The largest contributors of foreign tourists to Nepal are India (16%), China (12%), the United States (8%), Sri Lanka (7%) and the United Kingdom (6%).[l][211]

    Nepal 's tourism sector officially opened for westerners in 1951 but remained seriously hindered by a lack of proper planning and investment, continuous political instability. Once a popular final destination at the end of the hippie trail with legalised marijuana and hashish shops in Kathmandu, Nepali tourism was at its lowest during the civil war in the 1990s. Enthused by the upsurge since the peace process began, Nepal aims to welcome two million tourists in 2020, double the 2018 figures, via the concerted Visit Nepal 2020 initiative. With a lack of proper facilities for high-end tourism termed the "infrastructure bottleneck", the flag carrier in shambles and blacklisted by the developed countries, and with only a small number of popular destinations properly developed and marketed, the goal is considered too ambitious. The home-stay tourism, in which cultural and eco-tourists stay as paying guests in the homes of indigenous people, considered a more equitable and viable means of developing the sector, has seen some success.[214] Lumbini and other Hindu and Buddhist pilgrimages including Pashupati temple, Swayambhu and Boudhanath in Kathmandu, Muktinath in Mustang, and Janakpurdham, the capital of the ancient kingdom of Videha and home to goddess Sita, are among the premier destinations for the development of spiritual, religious and cultural tourism. Diversifying eco-tourism by developing and marketing less popular national parks, trekking routes and mountains are considered essential for the growth of the industry.[213]

    Foreign employment The rate of unemployment and underemployment exceeds half of the working - age population. [215] Thus, many Nepali citizens move to other countries in search of work.Destinations include India, the GCC countries, Malaysia, South Korea and Japan.In addition, thousands of well - educated people migrate to developed countries, like the United States, Australia and the UK
    for higher studies and work, half of whom never
    return.Most of the workers employed in India, the middle east and East Asia are unskilled labourers from very poor families.Many of them are swindled by the manpower companies and sent to exploitative employers or war - ridden countries under fraudulent contracts, in breach of government restrictions.In an extreme such
    case, in 2004, 12 Nepalese migrant workers travelling to an American base in Iraq through a third country, were kidnapped by terrorists and executed on video. [216][217] Nepalese workers have their passports seized upon arrival by their employer and only returned when they are allowed to
    return by the employer 's consent. Due to Nepal'
    s failure to secure rights
    for its workers, most Nepalese don 't get paid minimum wage,[218] and many are forced to return without being paid all or part of the wages.[219] Many Nepalese work in extremely unsafe conditions; an average of two Nepalese workers die each day.[220] The proportion of women participation in foreign employment is increasing. Due to restrictions placed on women, many are forced to depend on traffickers to get out of the country, and end up victims of violence and abuse.[221] Many Nepalese are believed to be working under slavery-like conditions, and Nepal spends billions of rupees rescuing stranded workers, on remuneration to the indebted families of the dead and legal costs for those who get jailed in foreign countries.[222][223]

    Foreign employment is the largest source of foreign exchanges
    for Nepal.Due to millions of missing men, women have been left to attend to the farm and livestock, take care of the families, and raise children, all by themselves.Nepal 's population growth rate has remained lower than projected, in part, due to the same. Although millions of families have been raised out of poverty by foreign employment, due to a lack of entrepreneurial skills, the earnings that workers bring back to Nepal are mostly spent on real estate and consumption.[224][223]

    Infrastructure Main article: List of FM radio stations in Nepal Main article: List of newspapers in Nepal

    Middle Marsyangdi Hydroelectric Dam.Nepal has significant potential to generate hydropower, which it plans to
    export across South Asia The bulk of energy in Nepal comes from biomass(80 % ) and imported fossil fuels(16 % ). [225] Most of the final energy consumption goes to the residential sector(84 % ) followed by transport(7 % ) and industry(6 % ), but the transport and industry sectors have been expanding rapidly in recent years. [225] Except
    for some lignite deposits, Nepal has no known oil, gas or coal deposits. [225] All commercial fossil fuels(mainly oil, LPG and coal) are imported, spending 129 % of the country 's total export revenue.[226] Only about 1% of the energy need is fulfilled by electricity.[225] The perennial nature of Nepali rivers and the steep gradient of the country'
    s topography provide ideal conditions
    for the development of hydroelectric projects.Estimates put Nepal 's economically feasible hydropower potential at approximately 42,000 MW.[225] However, Nepal has been able to exploit only about 1,100 MW. As most of it is generated from run-of-river (ROR) plants, the actual power produced is much lower in the dry winter months when peak demand can reach as high as 1,200 MW, and Nepal needs to import as much as 650 MW from India to meet the demands.[227] Nepal aims to bring enough hydropower projects into operation by the end of 2019 to go into surplus, but major hydropower projects have continued to suffer delays and setbacks.[228][229][230] Nepal'
    s electrification rate(76 % ) is comparable to that of other countries in the region but there is significant disparity between the rural(72 % ) and urban(97 % ) areas. [225] The position of the power sector remains unsatisfactory because of high tariffs, high system losses, high generation costs, high overheads, over staffing, and lower domestic demand. [231]

    Nepal remains isolated from the world 's major land, air and sea transport routes although, within the country, aviation is in a better state, with 47 airports, 11 of them with paved runways;[232] flights are frequent and support a sizeable traffic. The hilly and mountainous terrain in the northern two-thirds of the country has made the building of roads and other infrastructure difficult and expensive. As of 2016, there were just over 11,890 km (7,388 mi) of paved roads, and 16,100 km (10,004 mi) of unpaved roads, and just 59 km (37 mi) of railway line in the south.[232] As of 2018, all district headquarters (except Simikot) had been connected to the road network.[167] Most of the rural roads are not operable during the rainy season; even national highways regularly become inoperable.[233] Nepal depends almost entirely on foreign assistance from countries like China, India and Japan, for building, maintenance and expansion of the road network. The only practical seaport of entry for goods bound for Kathmandu is Kolkata in West Bengal state of India. The national carrier, Nepal Airlines, is in poor shape due to mismanagement and corruption, and has been blacklisted by the EU.[234] Internally, the poor state of development of the road system makes access to markets, schools, and health clinics a challenge.[207]

    According to the Nepal Telecommunication Authority MIS August 2019 report, voice telephony subscription rate was at 2.70 % of total population
    for fixed phones and 138.59 %
    for mobile; 98 % of all voice telephony was through mobile phones. [235] Similarly,
    while an estimated 14.52 % had access to fixed broadband, an additional 52.71 % were accessing the internet using their mobile data subscriptions; almost 15 million of them with 3 G or better. [235] The mobile voice telephony and broadband market was dominated by two telecommunications companies, the state - owned Nepal Telecom(55 % ) and the private multinational, Ncell(40 % ). [235] Of the 21 % market share enjoyed by fixed broadband, around 25 % was again shared by Nepal Telecom, with the rest going to the private Internet Service Providers. [235] Although there is high disparity in penetration rate between the rural and urban areas, mobile service has reached 75 districts of the country covering 90 % of land area, and broadband access is expected to reach 90 % of the population by 2020.[232]

    As of 2019, the state operates three television stations as well as national and regional radio stations.There are 117 private TV channels and 736 FM radio stations licensed
    for operation, at least 314 of them, community radio stations. [232] According to the 2011 census, the percentage of households possessing radio was 50.82 % , television 36.45 % , cable TV 19.33 % , and computer 7.23 % . [8] According to the Press Council Nepal classification, as of 2017 of the 833 publications producing original content, ten national dailies and weeklies are rated A + class. [236] In 2019, Reporters Without Borders ranked Nepal at 106 th in the world in terms of press freedom. [237]

    Demographics Main article: Demographics of Nepal The citizens of Nepal are known as Nepali or Nepalese.The Nepali are descendants of three major migrations from India, Tibet and North Burma, and the Chinese province of Yunnan via Assam.Among the earliest inhabitants were the Kirat of the eastern region, Newars of the Kathmandu Valley, aboriginal Tharus of the Terai plains and the Khas Pahari people of the far - western hills.Despite the migration of a significant section of the population to the Terai in recent years, the majority of Nepalese still live in the central highlands, and the northern mountains are sparsely populated.

    Nepal is a multicultural and multiethnic country, home to 126 distinct ethnic groups, speaking 123 different mother tongues and following a number of indigenous and folk religions in addition to Hinduism, Buddhism, Islam and Christianity. [8] According to the 2011 census, Nepal 's population was 26.5 million, almost a threefold increase from nine million in 1950. From 2001 to 2011, the average family size declined from 5.44 to 4.9. The census also noted some 1.9 million absentee people, over a million more than in 2001; most are male labourers employed overseas. This correlated with the drop in sex ratio from 94.41 as compared to 99.80 for 2001.[8] The annual population growth rate was 1.35% between 2001 and 2011, compared to an average of 2.25% between 1961 and 2001; also attributed to the absentee population.[238]

    Nepal is one of the ten least urbanised, and the ten fastest urbanising countries in the world.As of 2014, an estimated 18.3 % of the population lived in urban areas.Urbanisation rate is high in the Terai, doon valleys of the inner Terai and valleys of the middle hills, but low in the high Himalayas.Similarly, the rate is higher in central and eastern Nepal compared to further west. [239] The capital, Kathmandu, nicknamed the "City of temples", is the largest city in the country and the cultural and economic heart.Other large cities in Nepal include Pokhara, Biratnagar, Lalitpur, Bharatpur, Birgunj, Dharan, Hetauda and Nepalgunj.Congestion, pollution and drinking water shortage are some of the major problems facing the rapidly growing cities, most prominently the Kathmandu Valley.

    Language Main article: Languages of Nepal

    A Nepalese Tibetan monk Nepal 's diverse linguistic heritage stems from three major language groups: Indo-Aryan, Tibeto-Burman, and various indigenous language isolates. The major languages of Nepal (percent spoken as native language) according to the 2011 census are Nepali (44.6%), Maithili (11.7%), Bhojpuri (6.0%), Tharu (5.8%), Tamang (5.1%), Nepal Bhasa (3.2%), Bajjika (3%) and Magar (3.0%), Doteli (3.0%), Urdu (2.6%), Awadhi (1.89%), and Sunwar. Nepal is home to at least four indigenous sign languages.

    Descendent of Sanskrit, Nepali is written in Devanagari script.It is the official language and serves as lingua franca among Nepali of different ethnolinguistic groups.The regional languages Maithili, Awadhi and Bhojpuri are spoken in the southern Terai region; Urdu is common among Nepali Muslims.Varieties of Tibetan are spoken in and north of the higher Himalaya where standard literary Tibetan is widely understood by those with religious education.Local dialects in the Terai and hills are mostly unwritten with efforts underway to develop systems
    for writing many in Devanagari or the Roman alphabet.

    Religion Main article: Religion in Nepal

    Sadhus in Pashupatinath Temple The 2011 census reported that the religion with the largest number of followers in Nepal was Hinduism(81.3 % of the population), followed by Buddhism(9 % ); the remaining were Islam(4.4 % ), Kirant(3 % ), Christianity(1.3 % ) and other folk religions(0.4 % ).The prevalence of irreligion was reported to be at 0.5 % . [1] By percentage of population, Nepal has the largest population of Hindus in the world. [240] Nepal was officially a Hindu Kingdom until recently, and Shiva was considered the guardian deity of the country. [241] Although many government policies throughout history have disregarded or marginalised minority religions, Nepalese societies generally enjoy religious tolerance and harmony among all religions, with only isolated incidents of religiously - motivated violence. [242][243]

    Education Main article: Education in Nepal Nepal entered modernity in 1951 with a literacy rate of 5 % and about 10, 000 students enrolled in 300 schools. [244] By 2017, there were more than seven million students enrolled in 35, 601 schools. [245] The overall literacy rate(
      for population age 5 years and above) increased from 54.1 % in 2001 to 65.9 % in 2011. The male literacy rate was 75.1 % compared to the female literacy rate of 57.4 % .The highest literacy rate was reported in Kathmandu district(86.3 % ) and lowest in Rautahat(41.7 % ). [8] While the net primary enrolment rate was 74 % in 2005;
    [246] it had reached 97 % by 2017.[247] However, increasing access to secondary education(grades 9– 12) remains a major challenge, as evidenced by the low net enrolment rate of 57.5 % at this level. [248] Tertiary gross enrolment rate was only 12.4 % As of 2018.[248] By 2017, females had overtaken males in enrolment at all levels, compared to ten years earlier when male enrolment at the tertiary level was almost double that of females. [248] As of 2017, Nepal had eleven universities and four independent science academies. [245] Lack of proper infrastructures and teaching materials and a high student - to - teacher ratio, as well as politicisation of school management committees[249] and partisan unionisation among both students and teachers, [250] present a significant hurdle to progress.Free basic education is guaranteed in the constitution but the programme lacks funding
    for effective implementation. [251] Government has scholarship programs
    for girls, disabled and children of martyrs, of marginalised communities and the extremely poor. [252] The government also provides various scholarships
    for the students of marginalised communities, the poor and the meritorious,
    for higher education including medical and engineering studies. [253] Tens of thousands of Nepali students leave the country every year in search of better education and work, with half of them never returning. [254][255]

    Health and sanitation Main article: Health in Nepal

    Kunde Hospital in remote Himalayan region Public health and health care services in Nepal are provided by both the public and private sectors.Tap water is the main source of drinking water
    for 47.78 % of households, tube well / hand pump is the main source of drinking water
    for about 35 % of households,
    while spout, uncovered well / kuwa, and covered well / kuwa are the main source
    for 5.74 % , 4.71 % , and 2.45 % respectively. [8] Life expectancy at birth is estimated at 71 years as of 2017, 153 rd highest in the world, [256] up from 54 years in the 1990 s. [257] Two - thirds of all deaths are due to non - communicable diseases, 9 % due to injuries and 25 % due to communicable, maternal, neonatal and nutritional diseases.Heart disease is the leading cause of death; other major causes are injuries, diarrhoeal diseases, respiratory diseases and intracerebral haemorrhage. [258] HIV AIDS is the seventeenth leading cause of death. [257] Of the 32, 000 HIV infected, only 13, 000 are receiving antiretroviral treatment, As of 2018.[259] Diabetes and kidney disease are highly prevalent as well. [260] Sedentary lifestyle, imbalanced diet and malnutrition, consumption of tobacco and alcohol, and poor sanitation are among the driving forces behind poor health in Nepal. [260][258] Nepal has made tremendous progress in maternal and child health .95 % of children have access to iodised salt, and 86 % of children aged 6– 59 months receive Vitamin A prophylaxis. [261] Stunting has been reduced to 36 % in 2016 from 57 % in the 1996; underweight was reduced to 27 % from 42 % , and wasting to 10 % from 15 % , in the same period;
    [261] however, malnutrition, at 43 % among children under five, is extremely high. [262] Anaemia in women and children increased between 2011 and 2016, reaching 41 % and 53 % respectively. [262] Low birth weight is at 27 %
    while breastfeeding is at 65 % . [262] Nepal has reduced maternal mortality rate to 229[263] from 901 in 1990, [264] but is expected to fall short of the targeted 125 per 100, 000 live births by 2020.[263] Infant mortality per thousand live births is down to 32.2 compared to 139.8 in 1990.[265] Contraceptive prevalence rate is 53 % but the disparity rate between rural and urban areas is high due to a lack of awareness and easy access. [266]

    Progress in health is driven by strong government initiative in cooperation with NGOs and INGOs.Public health centres provide 72 essential medicines free of cost.In addition, the public health insurance plan initiated in 2016 which covers health treatments of up to Rs 50, 000
    for five members of a family,
    for a premium of Rs 2500 per year, has seen limited success, and is expected to expand. [267] By paying stipends
    for four antenatal visits to health centers and hospitalised delivery, Nepal decreased home - births from 81 % in 2006[264] to 41 % in 2016.[268] School meal programs have improved education as well as nutrition metrics among children. [269] Toilet building subsidies under the ambitious "one household-one toilet"
    program which aims to make the country open defecation free has seen toilet prevalence rate reach 99 % in 2019, from just 6 % in 1990.[270]

    Immigrants and refugees Nepal has a long tradition of accepting migrants and refugees. [271] In modern times, Tibetans and Bhutanese have constituted a majority of refugees in Nepal.Tibetan refugees began arriving in Nepal in 1959, [272] and many more cross into Nepal every year. [273] The Bhutanese Lhotsampa refugees began arriving in Nepal in the 1980 s and had reached more than 110, 000 by the 2000 s.Most of them have been resettled into third countries. [274] At the end of 2018, Nepal had a total of 20, 800 confirmed refugees, 64 % of them Tibetan and 31 % Bhutanese. [275] Recent arrivals among the refugees and asylum seekers, who are termed "urban refugees"
    as they live in apartments in the cities instead of refugee camps, include Ahmaddiya Muslims of Pakistan and Rohingya Muslims from Myanmar, but also some immigrants from Afghanistan, Bangladesh, Iraq, Pakistan, Somalia and other African countries, [276] and Sri Lanka. [277][278] Nepal does not recognise refugees other than the Bhutanese and Tibetans, terming them "illegal immigrants".However, the government facilitates third - country settlement
    for them, in cooperation with international partners. [279]

    About two thousand immigrants applied
    for work permit in Nepal in 2018 / 19, half of them Chinese.Nepal does not have a record of the number of Indian immigrant workers in Nepal as they do not require work permits to live and work in Nepal. [280] Indian government puts the number of non - resident Indians in Nepal at 600, 000.[281]

    Society and culture Main article: Culture of Nepal Society

    A Magar couple in their ethnic dress.Folklore is an integral part of Nepali society.Traditional stories are rooted in the reality of day - to - day life, tales of love, affection and battles as well as demons and ghosts and thus reflect local lifestyles, culture, and beliefs.Many Nepali folktales are enacted through the medium of dance and music.

    Most houses in the rural lowlands of Nepal are made up of a tight bamboo framework and walls of a mud and cow - dung mix.These dwellings remain cool in summer and retain warmth in winter.Houses in the hills are usually made of unbaked bricks with thatch or tile roofing.At high elevations construction changes to stone masonry and slate may be used on roofs.

    With 15 days a year, Nepal is the country that enjoys the least number of public holidays in the world. [282] The Nepali year begins in 1 st of Baisakh in official Hindu Calendar of the country, the Bikram Sambat, which falls in mid - April and is divided into 12 months.Saturday is the official weekly holiday.Main annual holidays include the Martyr 's Day (18 February), and a mix of Hindu and Buddhist festivals such as Dashain in autumn, Tihar in mid-autumn and Chhath in late autumn. During Swanti, the Newars perform the Mha Puja ceremony to celebrate New Year'
    s Day of the lunar calendar Nepal Sambat.Being a Secular country Nepal has holiday on main festivals of minority religions too. [283]

    Symbols Nepal 's national colour is crimson. Rhododendron in the national flower. Himalayan monal is the national bird and cow is the national animal. Volleyball is the national sport. Nepali is the national language. The Sayaun Thunga Phool Ka is the national anthem.

    The emblem of Nepal depicts the snowy Himalayas, the forested hills, and the fertile Terai, supported by a wreath of rhododendrons, national flag at the crest and in the foreground, a plain white map of Nepal with a man 's(right) and woman'
    s(left) right hands joining to signify gender equality just below it.At the bottom is the national motto of Nepal, a Sanskrit quote of patriotism attributed in Nepal folklore to Lord Rama, written in Devanagari script, which reads "Mother and motherland are greater than heaven".

    Nepal 's flag is the only national flag in the world that is not rectangular in shape.[284]

    The constitution of Nepal contains instructions
    for a geometric construction of the flag. [285] According to its official description, the red in the flag stands
    for victory in war or courage, and is also the colour of the rhododendron.The flag 's blue border signifies Nepalese people'
    s desire
    for peace.The moon on the flag is a symbol of the peaceful and calm nature of Nepali,
    while the sun represents the aggressiveness of Nepali warriors.The national flag was adopted in 1928 and then the sun and the moon had faces.In 1962 the faces were removed from both triangles. [286]

    The president is the symbol of national unity.The martyrs are the symbols of patriotism, prominent among them, four great martyrs - Gangalal Shrestha, Dharmabhakta Mathema, Sukraraj Shastri and Dasharath Chand.King Prithvi Narayan Shah, the founder of modern Nepal, is held in high regard and considered "Father of the Nation"
    by many. [287][288]

    Nepali commanders of the Anglo - Nepal war, Amar Singh Thapa, Bhakti Thapa, Balbhadra Kunwar and Ujir Singh Thapa are considered war heroes.A special designation of "National hero"
    has been conferred to 16 people from Nepalese history who made exceptional contributions to the prestige of Nepal; those include such diverse figures as Gautam Buddha, Sita, King Janaka, Araniko, Motiram Bhatta, Mahaguru Falgunanda and Pasang Lhamu Sherpa.

    Writers, especially poets, are venerated in Nepal and conferred special titles, such as first poet, great poet, national poet and poet par excellence, among others.

    Art and architecture Main article: Architecture of Nepal

    Nyatapola was erected by King Bhupatindra Malla in 1702 and is a major tourist attraction in the historical city of Bhaktapur.Historical kingdoms that existed in the Kathmandu valley are found to have made use of some clever technologies in numerous areas such as architecture, agriculture, civil engineering, water management, etc.The Gopals and Abhirs, who ruled the valley up until c .1000 BC, used temporary materials
    for construction such as bamboo, hay, timber, etc.The Kirat period(700 BC– 110 AD) employed the technology of brick firing as well as produced quality woolen shawls.Similarly, stupas, idols, canals, self - recharging ponds, reservoirs, etc.constructed during the Lichhavi era(110– 879 AD) are intact to this day, which manifests the ingenuity of traditional architecture.Moreover, the Malla period(1200– 1768 AD) saw an impressive growth in architecture, on par with its advanced contemporaries.An archetypal example of Malla architecture is Nyatapola, a five - storied, 30 - metre tall temple in Bhaktapur, which has strangely survived at least four major earthquakes, including the April 2015 Nepal earthquake. [289]

    Literature and the performing arts Main articles: Nepalese literature, Music of Nepal, and Cinema of Nepal Nepal 's literature was closely intertwined with that of the rest of South Asia until its unification into a modern kingdom. Literary works, which were written in Sanskrit by Brahmin priests educated and sometimes also based in Varanasi, included religious texts and other fantasies involving kings, gods and demons.[290] The oldest extant Nepali language text is dated to the 13th century but except for the epigraphic material, Nepali language literature older than the 17th century haven'
    t been found.However, Newar literature dates back almost 500 years. [288] The modern history of Nepali literature begins with Bhanubhakta Acharya(1814 - 1868), who
    for the first time composed major and influential works in Nepali, the language accessible to the masses, most prominently, the Bhanubhakta Ramayana, a translation of the ancient Hindu epic. [290] By the end of the nineteenth century, Motiram Bhatta had published print editions of the works of Acharya, and through his efforts, single - handedly popularised and propelled Nepali language literature into modernity. [288] By the mid - twentieth century, Nepali literature was no longer limited to the Hindu literary traditions.Influenced by western literary traditions, writers in this period started producing literary works addressing the contemporary social problems, [291]
    while many others continued to enrich Nepali poetic traditions with authentic Nepali poetry.Newar literature also emerged as a premier literary tradition.After the advent of democracy in 1951, Nepali literature flourished.Literary works in many other languages began to be produced.Nepali literature continued to modernise, and in recent years, has been strongly influenced by the post civil - war Nepali experience as well as global literary traditions. [292][293][294][288]

    Maruni, Lakhey and Tamang Selo are some examples of the traditional Nepali music and dance in the hilly regions of Nepal.

    Nepali film industry is known as "Kollywood". [295]

    Nepal Academy is the foremost institution
    for the promotion of arts and culture in Nepal, established in 1957.[288]

    Clothing

    A Nepali man in Daura - Suruwal, coat and Dhaka topi, displays the bhoto during the Bhoto Jatra festival.The most widely worn traditional dress in Nepal,
    for both women and men, from ancient times until the advent of modern times, was draped. [296] For women, it eventually took the form of a sari, a single long piece of cloth, famously six yards long, and of width spanning the lower body. [296] The sari is tied around the waist and knotted at one end, wrapped around the lower body, and then over the shoulder. [296] In its more modern form, it has been used to cover the head, and sometimes the face, as a veil, [296] particularly in the Terai.It has been combined with an underskirt, or the petticoat, and tucked in the waistband
    for more secure fastening.It is worn with a blouse, or cholo, which serves as the primary upper - body garment, the sari 's end, passing over the shoulder, now serving to obscure the upper body'
    s contours, and to cover the midriff. [296] Cholo - sari has become the attire of choice
    for formal occasions, official environs and festive gatherings.In its more traditional form, as part of traditional dresses and as worn in daily life
    while performing household chores or labour, it takes the form of a fariya or gunyu, usually shorter than a sari in length as well as breadth, and all of it wrapped around the lower body.

    For men, a similar but shorter length of cloth, the dhoti, has served as a lower - body garment. [297] It too is tied around the waist and wrapped. [297] Among the Aryans, it is also wrapped once around each leg before being brought up through the legs to be tucked in at the back.Dhoti or its variants, usually worn over a langauti, constitute the lower - body garment in the traditional clothing of Tharus, Gurungs and Magars as well as the Madhesi people, among others.Other forms of traditional apparel that involve no stitching or tailoring are the Patukas(a length of cloth wrapped tightly over the waist by both sexes as a waistband, a part of most traditional Nepali costumes, usually with a Khukuri tucked into it when worn by men), scarves like Pachhyauras and majetros and shawls like the Newar Ga and Tibetan khata, Ghumtos(the wedding veils) and various kinds of turbans(scarves worn around the head as a part of a tradition, or to keep off the sun or the cold, [297] called a Pheta, Pagri or Sirpau).

    Until the beginning of the first millennium CE, the ordinary dress of people in South Asia was entirely unstitched. [298] The arrival of the Kushans from Central Asia, circa 48 CE, popularised cut and sewn garments in the style of Central Asia. [298] The simplest form of sewn clothing, Bhoto(a rudimentary vest), is a universal unisex clothing
    for children throughout Nepal, and traditionally the only clothing children wear until they come of age and are given adult garb, sometimes in a ceremonial rite of passage, such as the gunyu - choli ceremony
    for Hindu girls.Men
    continue to wear bhoto through adulthood.Upper body garment
    for men is usually a vest such as the bhoto, or a shirt similar to the Kurta, most prominent of which is Daura, a closed - necked double - breasted long shirt with five pleats and eight strings that serve to tie it around the body.Suruwal, simply translated as a pair of trousers, is an alternative to and, more recently, replacement
    for dhoti, kachhad(Magars) or Lungi(Tharus); it is traditionally much wider above the knees but tapers below the knees to fit tightly at the ankles, is tied to the waist with a drawstring and does not use the modern inventions like elastic waistband, zippers, hooks or buttons.Modern cholos worn with sarees are usually half - sleeved, and single - breasted fastened in the front or front - covering tied at the back; they usually do not cover the midriff.The traditional one called the chaubandi cholo, similar to the daura, is full - sleeved, double - breasted with pleats and strings to tie it around the body, and extends down to cover the midriff, usually reaching up to or covering the patuka.

    Daura - Suruwal and Gunyu - Cholo were the national dresses of Nepal
    for men and women respectively until 2011 when they were removed to eliminate favouritism. [299] Traditional dresses of many pahari ethnic groups comprise Daura - Suruwal or similar, with patuka, a dhaka topi and a coat
    for men, and Gunyu - cholo or similar, with patuka and with or without some kind of scarf
    for women.For many other groups, men 's traditional dresses consist of a shirt or a vest, paired with a dhoti, kachhad or lungi. In the high himalayas, the traditional dresses are largely influenced by Tibetan culture. Sherpa women wear the chuba with the pangi (apron) apron, while Sherpa men wear shirts with stiff high collar and long sleeves called tetung under the chuba. Tibetan Xamo Gyaise hats of the Sherpas, dhaka topi of pahari men and tamang round caps are among the more distinctive headwears, while many ethnic groups wear some kind of scarf on their head. The bhangra, worn over the coat by the Gurung people is designed to function as a rucksack. Surke thaili, a small cloth purse with drawstring mechanism, usually accompanies the traditional dresses, as most of them lack pockets.

    Men wear minimal jewellery.Married Hindu women wear tika, sindur, pote and red bangles.Jewellery of gold and silver are common; precious stones are also used.Gold jewellery includes Mangalsutras and tilaharis worn with the pote by the Hindus, Samyafung(a huge gold flower worn on the head) and Nessey(huge flattened gold earrings) worn by the Limbus, and Sirphuli, Sirbandhi and Chandra worn by the Magars, among others.Tharu women can wear as much as six kilos of silver in jewellery at a time, which includes Mangiya worn on the head, tikuli the forehead, and kanseri and tikahamala worn around the neck. [300] Among some indigenous groups, particularly women, traditionally, tattoo their bodies and sometimes their faces.

    In the last 50 years, fashions have changed a great deal in Nepal.Increasingly, in urban settings, the sari is no longer the apparel of everyday wear, transformed instead into one
    for formal occasions.The traditional kurta suruwal is rarely worn by younger women, who increasingly favour jeans.The dhoti has largely been reduced to the liturgical vestment of shamans and Hindu priests.

    Cuisine Main article: Nepalese cuisine

    A dal - bhat thali with boiled rice, lentil soup, fried leafy greens, vegetable curry, yoghurt, papad and vegetable salad Nepalese cuisine consists of a wide variety of regional and traditional cuisines.Given the range of diversity in soil type, climate, culture, ethnic groups, and occupations, these cuisines vary substantially from each other, using locally available spices, herbs, vegetables, and fruit. [301] Nepali foodways have been influenced by religion, in particular Hindu and Buddhist cultural choices and traditions.The Columbian exchange had brought the potato, the tomato, maize, peanuts, cashew nuts, pineapples, guavas, and most notably, chilli peppers, to South Asia.Each became staples of use. [302] The cereals grown in Nepal, their choice, times, and regions of planting, correspond strongly to the timing of Nepal 's monsoons,[303] and the variations in altitude. Rice and wheat are mostly cultivated in the terai plains and well-irrigated valleys, and maize, millet, barley and buckwheat in the lesser fertile and drier hills.[301][304]


    Momo dumplings with chutney The foundation of a typical Nepali meal is a cereal cooked in plain fashion, and complemented with flavorful savory dishes. [305] The latter includes lentils, pulses and vegetables spiced commonly with ginger and garlic, but also more discerningly with a combination of spices that may include coriander, cumin, turmeric, cinnamon, cardamon, jimbu and others as informed by culinary conventions. [305] In an actual meal, this mental representation takes the form of a platter, or thali, with a central place
    for the cooked cereal, peripheral ones, often in small bowls,
    for the flavorful accompaniments, and the simultaneous, rather than piecemeal, ingestion of the two in each act of eating, whether by actual mixing—
    for example of rice and lentils— or in the folding of one— such as bread— around the other, such as cooked vegetables. [305] Dal - bhat, centred around steamed rice, which is accompanied by lentil soup, vegetable curries, chutneys and pickle, [306] as well as dairy and sometimes meat, is the most common and prominent example.The unleavened flat bread made from wheat flour called chapati occasionally replaces the steamed rice, particularly in the Terai,
    while Dhindo, prepared by boiling corn, millet or buckwheat flour in water, continuously stirring and adding flour until thick, almost solid consistency is reached, is the main substitute
    for rice in the hills and mountains.Tsampa, flour made from roasted barley and naked barley, is the main staple in the high himalayas.Fermented, then sun - dried, leafy greens called Gundruk are both a delicacy and a vital substitute
    for fresh vegetables in the winter
    for many, throughout Nepal. [304]

    A notable feature of Nepali food is the existence of a number of distinctive vegetarian cuisines, each a feature of the geographical and cultural histories of its adherents. [307] The appearance of ahimsa, or the avoidance of violence toward all forms of life in many religious orders early in South Asian history, especially Upanishadic Hinduism, Buddhism and Jainism, is thought to have been a notable factor in the prevalence of vegetarianism among a segment of Nepal 's Hindu and Buddhist populations, as well as among Jains.[307] Among these groups, strong discomfort is felt at thoughts of eating meat.[308] Though per capita meat consumption is low in Nepal, the proportion of vegetarianism is not high as that of India, due to the prevalence of Shaktism, of which animal sacrifice is a prominent feature.


    Samayabaji(Newar cuisine) Although overshadowed by the Indian and Tibetan cuisines in the global consciousness, [309] Nepali cuisines possess their own distinctive qualities to distinguish these hybrid cuisines from both their northern and southern neighbours. [306] Nepali cuisines, with generally tomato - based, leaner curries, are lighter than their cream - based Indian counterparts, and Nepali momo dumplings are heavily spiced compared to their northern counterparts. [306] Newar cuisine, one of the richest and most influential in Nepal, is more elaborate and diverse than most, as Newar culture developed in the highly fertile Kathmandu valley, and Newar people have historically been more well - travelled and affluent than others. [301] A typical Newar cuisine can comprise more than a dozen dishes of cereals, meat, vegetable curries, chutneys and pickles.Kwanti(sprouted beans soup), chhwela(ground beef), chatamari, (rice flour crepe), bara(fried lentil cake), kachila(marinated raw minced beef), samaybaji(centred around flattened rice), lakhaamari and yomuri are among the more widely recognised. [306][310] Juju dhau, a sweet yoghurt originating in Bhaktapur, is also famous. [306] Thakali cuisine is another well - known food tradition which seamlessly melds the Tibetan and the Indian with variety in ingredients, especially the herbs and spices. [301] Thukpa, a ramen dish originating in Tibet, is enlivened with beef and vegetables and spiced with Garam masala and chili peppers. [306] In the Terai, Bagiya is a rice flour dumpling with sweets inside, popular among the Tharu and Maithil people.Various communities in the Terai make sidhara(sun - dried small fish mixed with taro leaves) and biriya(lentil paste mixed with taro leaves) to stock
    for the monsoon floods. [301] Selroti, kasaar, fini and chaku are among the sweet delicacies.Rice pulau or sweet rice porridge called kheer are usually the main dish in feasts. [304] Tea and buttermilk(fermented milk leftover from churning butter from yoghurt) are common non - alcoholic drinks.Almost all janajati communities have their own traditional methods of brewing alcohol.Raksi(traditional distilled alcohol), jaand(rice beer), tongba(millet beer) and chyaang are the most well - known.In the hilly regions, various janajatis hunt stream - frogs known as Pahas. [311] In the eastern hills, Kirat people make yangben faska by cooking lichen known as yangben with pork. [301] In the Terai, Tharu cuisines includes dishes of rats, mussels and snails. [301][312] Rats are also hunted and eaten by the Musahar community in the Terai, whose name literally translates to rat - eaters.

    Sports and recreation Main article: Sports in Nepal

    Nepali children playing a variant of knucklebones with pebbles.Nepali indigenous sports, like dandi biyo and kabaddi which were considered the unofficial national sports until recently, [313] are still popular in rural areas. [314] Despite various efforts, standardisation and development of dandi biyo has not been achieved, [315][316]
    while Kabaddi, as a professional sport, is still in its infancy in Nepal. [317] Bagh - chal, an ancient board game that 's thought to have originated in Nepal, can be played on chalk-drawn boards, with pebbles, and is still popular today.[318][319] Ludo, snakes and ladders and carrom are popular pastimes.[320] Chess is also played.[314] Volleyball was declared as the national sport of Nepal in 2017.[313] Popular children'
    s games include versions of tag, [314] knucklebones, [m][314] hopscotch, [n] Duck, duck, goose[314] and lagori,
    while marbles, [314] top, [o] hoop rolling and gully cricket are also popular among boys.Rubber bands, or ranger bands cut from bike tyres, make a multi - purpose sporting equipment
    for Nepali children, which may be bunched or chained together, and used to play dodgeball, cat 's cradle, jianzi[p][314] and a variety of skipping rope games.[314]


    Nepali cricket fans are renowned
    for an exceptionally enthusiastic support of their national team. [321][322] Football and cricket are popular professional sports. [323] Nepal is competitive in football in the South Asia region but has never won the SAFF championships, the regional tournament. [324][325] It usually ranks in the bottom quarter in the FIFA world rankings. [326] Nepal has had success in cricket and holds the elite ODI status, [327][328] consistently ranking in the Top 20 in the ICC ODI and T20I rankings. [329][330] Nepal has had some success in athletics and martial arts, having won many medals at the South Asian Games and some at the Asian games. [331] Nepal has never won an olympic medal. [332] Sports like basketball, volleyball, futsal, wrestling, competitive bodybuilding[332][333] and badminton are also gaining in popularity. [314] Women in football, cricket, athletics, martial arts, badminton and swimming have found some success. [334][332] Nepal also fields players and national teams in several tournaments
    for the differently abled, most notably in men 's[335] as well as women'
    s blind cricket. [336]

    The only international stadium in the country is the multi - purpose Dasarath Stadium where the men and women national football teams play their home matches. [337] Since the formation of the national team, Nepal has played its home matches of cricket at Tribhuvan University International Cricket Ground. [338] Nepal police, Armed police force and Nepal army are the most prolific producers of national players, and aspiring players are known to join armed forces,
    for the better sporting opportunities they can provide. [339][340] Nepali sports is hindered by a lack of infrastructure, [341] funding, [335] corruption, nepotism and political interference. [331][342][343] Very few players are able to make a living as professional sportspeople. [337]

    See also flag Nepal portal Outline of Nepal Notes ^
    The entire territory controlled by the monarch seated in Kathmandu at any given time would also be referred to as Nepal.Thus, at times, only the Kathmandu valley was considered Nepal
    while at other times, Nepal would encompass an area comparable to and largely overlapping with the modern state of Nepal[23] ^
    The word pala in Pali language means to protect.Consequently, Nepala translates to protected by Ne. ^
    Nepalamahatmya, of 30 chapters about the Nepal Tirtha(pilgrimage) region, is a regional text that claims to be a part of the Skanda Purana, the largest Mahāpurāṇa. ^
    This trichotomy is a prominent feature of Nepali discourse and is represented in the Emblem of Nepal, with blue and white peaks signifying Himal, green hills below them signifying Pahad and the yellow strip at the bottom signifying the Terai belt. ^
    198 ecological types were first proposed in 1976, which was further revised and reduced to 118, which was further reduced by IUCN to 59 in 1998, which was further reduced to 36 in 2002. As this issue has yet to be settled, the 35 - forest - type classification is generally preferred to the ecological categorisation. [72] ^
    According to the 2019 IUCN red list, two species of mammals, one bird species and three amphibian species are endemic to Nepal. [85] ^
    See Kot massacre and Bhandarkhal massacre ^
    later known as Janga Bahadur Rana ^
    On the flip side, same sex marriage with foreign nationals occurring in a jurisdiction that recognises same - sex marriage is now recognised in Nepal,
    for eligibility to obtain a "non-tourist visa"
    as dependent of a Nepali citizen, by verdict of the Supreme Court in 2017, as the laws don 't make sex-specific distinction in provisions relating to the rights of foreign nationals married to Nepali citizens.[130] ^
    October 2019, IMF update, excludes Somalia and Syria 
     '''

**Notice the word Nepal in code**

In [0]:
import textacy

In [21]:
# Parse the document with spaCy
doc = nlp(text)

# Extract semi-structured statements
statements = textacy.extract.semistructured_statements(doc, "Nepal")

for statement in statements:
    subject, verb, fact = statement
    print(f" - {fact}")

 - a multiethnic country with Nepali as the official language.


 - also a member of the Non-Aligned Movement and the Bay of Bengal Initiative
 - a polished form of Newar, the name of the indigenous people of Kathmandu valley
 - of roughly trapezoidal shape, about 800 kilometres(500 mi) long and 200
 - a parliamentary republic with a multi - party system
 - one of the major contributors to UN peacekeeping missions
 - the first South Asian country to establish diplomatic relations with Israel, and the countries enjoy a strong relationship.[155
 - also a member of WTO, BIMSTEC and ACD, among others
 - one of the major contributors to the UN Peacekeeping Missions having contributed more than 119,000 personnel to 42 missions since 1958.[175
 - one of the fastest growing economies in the world
 - a member of WTO since 23 April
 - home to eight of the fourteen eight-thousanders, all among the ten tallest mountains in the world
 - able to exploit only about 1,100 MW
 - one of the ten least ur

In [0]:

# Extract noun chunks that appear
noun_chunks = textacy.extract.noun_chunks(doc, min_freq=3)

# Convert noun chunks to lowercase strings
noun_chunks = map(str, noun_chunks)
noun_chunks = map(str.lower, noun_chunks)

# Print out any nouns that are at least 2 words long
for noun_chunk in set(noun_chunks):
    if len(noun_chunk.split(" ")) > 1:
        print(noun_chunk)

main articles
high himalayas
supreme court
other laws
traditional dresses
nepali literature
lower body
foreign employment
literary works
recent years
foreign nationals
executive power
political parties
nepal army
main article
constituent assembly
2011 census
central asia
nepal police
dhaka topi
south asia
united kingdom
90 %
indian plate
united states
gautam buddha
people 's republic
prime minister
provincial assembly
seven provinces
mount everest
king gyanendra
main source
national assembly
federal level
ruling party
nepali congress
other countries
kathmandu valley
terai plains
civil war
new constitution
local governments
daura - suruwal
